In [ ]:
import pandas as pd
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from matplotlib.ticker import MultipleLocator
from cinnabar import stats
import re

# Functions

In [ ]:
def calculate_rmse(predictions, targets):
    """Calculates RMSE using sqrt(mean((predicted_vals - target_vals)^2))"""
    return np.sqrt(sklearn.metrics.mean_squared_error(targets, predictions))

def calculate_mae(predictions, targets):
    """Calculates MAE using mean(abs(predicted_vals - target_vals))"""
    # mae = np.mean(np.absolute(predictions - targets))
    # return mae
    return sklearn.metrics.mean_absolute_error(targets, predictions)

def specific_pattern_match(cell, pattern):
    if pd.isna(cell):
        return False
    return bool(re.search(pattern, str(cell)))

def arr_to_kJ2kcal(arr):
    from scipy.constants import calorie
    kJ2kcal = 1 / calorie
    return arr * kJ2kcal


# Experimental data

In [ ]:
### EXPERIMENTAL ITC RESULTS
am_exp_ddg_kcal = {"v14g": 1.13, "t16g": 3.09, "q18g": 0.89, "i19g": 4.09, "t16g_i19g": 3.92}
nutlin_exp_ddg_kcal = {"v14g": 0.11, "t16g": 0.20, "q18g": 0.20, "i19g": 0.35, "t16g_i19g": 0.20}

am_exp_ddg_kcal_df = pd.DataFrame.from_dict(am_exp_ddg_kcal, orient="index", columns=["ddG (kcal/mol)"])
nutlin_exp_ddg_kcal_df = pd.DataFrame.from_dict(nutlin_exp_ddg_kcal, orient="index", columns=["ddG (kcal/mol)"])

am_exp_ddg_kcal_df, nutlin_exp_ddg_kcal_df

## EQ 0 to 100 % (Forward Convergence)

### Plotting

In [ ]:
am_df_full = pd.read_csv('eq_fep_analysed_results_0_to_100/am_df_full.csv')
apo_df_full = pd.read_csv('eq_fep_analysed_results_0_to_100/apo_df_full.csv')
nutlin_df_full = pd.read_csv('eq_fep_analysed_results_0_to_100/nutlin_df_full.csv')

In [ ]:
am_df_full


In [ ]:
temporal_rmse = {}
temporal_rmse_lower = {}
temporal_rmse_upper = {}
temporal_mae = {}
temporal_std = {}
temporal_am_pred_ddg_std = []
temporal_nutlin_pred_ddg_std = []

time_start_ns = 0 # ns

for time_end_ns in range(10, 110, 10):
    time_end = time_end_ns * 1000 # ns to ps


    # need to use the pandas query method to filter the dataframes because saving to csv did not save the multiindex
    am_open_slice_results_df = am_df_full[(am_df_full['time_start'] == 0) & (am_df_full['time_end'] == time_end)]
    apo_closed_slice_results_df = apo_df_full[(apo_df_full['time_start'] == 0) & (apo_df_full['time_end'] == time_end)]
    nutlin_closed_slice_results_df = nutlin_df_full[(nutlin_df_full['time_start'] == 0) & (nutlin_df_full['time_end'] == time_end)]

    
    am_pred_ddg = am_open_slice_results_df["dG avg. (kcal/mol)"] - apo_closed_slice_results_df["dG avg. (kcal/mol)"]
    nutlin_pred_ddg = nutlin_closed_slice_results_df["dG avg. (kcal/mol)"] - apo_closed_slice_results_df["dG avg. (kcal/mol)"]

    am_pred_ddg_std = np.sqrt(am_open_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std = np.sqrt(nutlin_closed_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)


    # create separate dataframes for keeping track of time depedenent std and the mutant names
    am_pred_ddg_std_timed = deepcopy(am_open_slice_results_df)
    nutlin_pred_ddg_std_timed = deepcopy(nutlin_closed_slice_results_df)

    am_pred_ddg_std_timed["ddg std. (kcal/mol)"] = np.sqrt(am_open_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std_timed["ddg std. (kcal/mol)"] = np.sqrt(nutlin_closed_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)

    # drop dG avg. (kcal/mol) column
    am_pred_ddg_std_timed.drop(columns=["dG avg. (kcal/mol)"], inplace=True)
    nutlin_pred_ddg_std_timed.drop(columns=["dG avg. (kcal/mol)"], inplace=True)

    temporal_am_pred_ddg_std.append(am_pred_ddg_std_timed)
    temporal_nutlin_pred_ddg_std.append(nutlin_pred_ddg_std_timed)


    # am_pred_ddg_std, nutlin_pred_ddg_std

    # rename columns
    am_pred_ddg = am_pred_ddg.rename("ddG avg. (kcal/mol)")
    am_pred_ddg_std = am_pred_ddg_std.rename("ddG std. (kcal/mol)")
    nutlin_pred_ddg = nutlin_pred_ddg.rename("ddG avg. (kcal/mol)")
    nutlin_pred_ddg_std = nutlin_pred_ddg_std.rename("ddG std. (kcal/mol)")

    am_pred_ddg_df = pd.DataFrame(am_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    am_pred_ddg_std = pd.DataFrame(am_pred_ddg_std, columns=["ddG std. (kcal/mol)"])
    nutlin_pred_ddg_df = pd.DataFrame(nutlin_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    nutlin_pred_ddg_std = pd.DataFrame(nutlin_pred_ddg_std, columns=["ddG std. (kcal/mol)"])

    # reindex the dataframes to match the experimental data
    am_pred_ddg_df.reindex(am_exp_ddg_kcal_df.index)
    am_pred_ddg_std.reindex(am_exp_ddg_kcal_df.index)

    nutlin_pred_ddg_df.reindex(nutlin_exp_ddg_kcal_df.index)
    nutlin_pred_ddg_std.reindex(nutlin_exp_ddg_kcal_df.index)


    # am_pred_ddg_df, nutlin_pred_ddg_df

    predicted_ddg_df = pd.concat([am_pred_ddg_df, nutlin_pred_ddg_df])
    predicted_ddg_std_df = pd.concat([am_pred_ddg_std, nutlin_pred_ddg_std])

    experimental_ddg_df = pd.concat([am_exp_ddg_kcal_df, nutlin_exp_ddg_kcal_df])

    # predicted_ddg_df, experimental_ddg_df

    # to numpy array
    predicted_ddg = predicted_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    predicted_ddg_std = predicted_ddg_std_df["ddG std. (kcal/mol)"].to_numpy()
    experimental_ddg = experimental_ddg_df["ddG (kcal/mol)"].to_numpy()


    # predicted_ddg, experimental_ddg

    rmse_partial = calculate_rmse(predicted_ddg, experimental_ddg)
    mae_partial = calculate_mae(predicted_ddg, experimental_ddg)

    # rmse_partial, mae_partial

    # convert to numpy arrays for plotting
    am_exp_ddg_kcal_np = am_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    am_pred_ddg_np = am_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    am_pred_ddg_std_np = am_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    nutlin_exp_ddg_kcal_np = nutlin_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_np = nutlin_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_std_np = nutlin_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    temporal_rmse[f"{time_start_ns}-{time_end_ns}"] = rmse_partial
    temporal_mae[f"{time_start_ns}-{time_end_ns}"] = mae_partial
    temporal_std[f"{time_start_ns}-{time_end_ns}"] = predicted_ddg_std.mean()


    ### PLOT OF Dominant Lid States



    fig, axs = plt.subplots(layout='tight')

    x = np.arange(-5, 10, 1)
    y = np.arange(-5, 10, 1)
    plt.rcParams.update({'font.size': 14})
    ax_x_lim, ax_y_lim = -2, 6
    padding_x = 0.15



    axs.fill_between(x, [i - 1 for i in y], [i + 1 for i in y], alpha=0.2, color='grey')
    axs.fill_between(x, [i - 0.5 for i in y], [i + 0.5 for i in y], alpha=0.2, color='grey')
    axs.errorbar(x=nutlin_exp_ddg_kcal_np, y=nutlin_pred_ddg_np, yerr=nutlin_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#6fc1dcff', ecolor='black', label='Nutlin-3a')
    axs.errorbar(x=am_exp_ddg_kcal_np , y=am_pred_ddg_np, yerr=am_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#F07167', ecolor='black', label='AM-7209')
    axs.legend(loc='upper left')
    axs.set_ylim(ax_x_lim, ax_y_lim)
    axs.set_xlim(ax_x_lim, ax_y_lim)
    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    axs.set_ylabel("Predicted $\Delta \Delta G$ (kcal/mol)")
    axs.set_xlabel("Experimental $\Delta \Delta G$ (kcal/mol)")
    axs.set_box_aspect(1)




    axs.annotate('V14G', (nutlin_exp_ddg_kcal_np[0], nutlin_pred_ddg_np[0]), size=12, xytext=(-45, -5), textcoords="offset pixels")
    axs.annotate('T16G', (nutlin_exp_ddg_kcal_np[1], nutlin_pred_ddg_np[1]), size=12, xytext=(15, -6), textcoords="offset pixels")
    axs.annotate('Q18G', (nutlin_exp_ddg_kcal_np[2], nutlin_pred_ddg_np[2]), size=12, xytext=(-45, -5), textcoords="offset pixels")
    axs.annotate('I19G', (nutlin_exp_ddg_kcal_np[3], nutlin_pred_ddg_np[3]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (nutlin_exp_ddg_kcal_np[4], nutlin_pred_ddg_np[4]), size=12, xytext=(-80, -5), textcoords="offset pixels")

    axs.annotate('V14G', (am_exp_ddg_kcal_np [0], am_pred_ddg_np[0]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G', (am_exp_ddg_kcal_np [1], am_pred_ddg_np[1]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('Q18G', (am_exp_ddg_kcal_np [2], am_pred_ddg_np[2]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('I19G', (am_exp_ddg_kcal_np [3], am_pred_ddg_np[3]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (am_exp_ddg_kcal_np [4], am_pred_ddg_np[4]), size=12, xytext=(15, -5), textcoords="offset pixels")

    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    # from matplotlib.ticker import FormatStrFormatter

    # axs.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    # axs.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    textstr1 = f"RMSE:      {rmse_partial:.2f}     MAE:      {mae_partial:.2f}\n"
    textstr = textstr1 
    #title = r'MDM2 Dominant Lid States Only (BSS Inputs - Mixture of Replicas)'
    title = f'MDM2 EQ. FEP Dominant State Lid Mutations (N = {len(experimental_ddg)})'
    # bootstrap statistics
    res_rmse = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='RMSE')
    print(res_rmse)
    res_mae = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='MUE')
    res_rho = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='rho')

    # append lower and upper bounds to the dictionary
    temporal_rmse_lower[f"{time_start_ns}-{time_end_ns}"] = res_rmse['low']
    temporal_rmse_upper[f"{time_start_ns}-{time_end_ns}"] = res_rmse['high']


    textstr = f"RMSE:        {res_rmse['mle']:.2f} [95%: {res_rmse['low']:.2f}, {res_rmse['high']:.2f}]\nMAE:        {res_mae['mle']:.2f} [95%: {res_mae['low']:.2f}, {res_mae['high']:.2f}]\n"
    full_title = f"{textstr} {title}"
    axs.set_title(full_title, family="monospace", loc='right')
    fig.set_size_inches(10,10)
    # plt.savefig(f'paper_eq_fep_plots/{time_start_ns}-{time_end_ns}ns.pdf', dpi=500, bbox_inches='tight')

## EQ 100 to 10 % (Forward Discard)

In [ ]:
am_df_reverse_full = pd.read_csv('eq_fep_analysed_results_100_to_10/am_df_full.csv')
apo_df_reverse_full = pd.read_csv('eq_fep_analysed_results_100_to_10/apo_df_full.csv')
nutlin_df_reverse_full = pd.read_csv('eq_fep_analysed_results_100_to_10/nutlin_df_full.csv')

In [ ]:
am_df_reverse_full

### Plotting

In [ ]:
sns.set_theme(style="ticks")
sns.set_context("paper", font_scale=2)


temporal_rmse = {}
temporal_rmse_lower = {}
temporal_rmse_upper = {}
temporal_mae = {}
temporal_std = {}
temporal_am_pred_ddg_std = []
temporal_nutlin_pred_ddg_std = []

time_end_ns = 100 # ns

for time_start_ns in range(10, 100, 10):
    time_start = time_start_ns * 1000 # ns to ps

    # need to use the pandas query method to filter the dataframes because saving to csv did not save the multiindex
    am_open_slice_results_df = am_df_reverse_full[(am_df_reverse_full['time_start'] == time_start) & (am_df_reverse_full['time_end'] == 100000)]
    apo_closed_slice_results_df = apo_df_reverse_full[(apo_df_reverse_full['time_start'] == time_start) & (apo_df_reverse_full['time_end'] == 100000)]
    nutlin_closed_slice_results_df = nutlin_df_reverse_full[(nutlin_df_reverse_full['time_start'] == time_start) & (nutlin_df_reverse_full['time_end'] == 100000)]

    
    time_end = time_end_ns * 1000 # ns to ps
    am_pred_ddg = am_open_slice_results_df["dG avg. (kcal/mol)"] - apo_closed_slice_results_df["dG avg. (kcal/mol)"]
    nutlin_pred_ddg = nutlin_closed_slice_results_df["dG avg. (kcal/mol)"] - apo_closed_slice_results_df["dG avg. (kcal/mol)"]

    am_pred_ddg_std = np.sqrt(am_open_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std = np.sqrt(nutlin_closed_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)

    # create separate dataframes for keeping track of time depedenent std and the mutant names
    am_pred_ddg_std_timed = deepcopy(am_open_slice_results_df)
    nutlin_pred_ddg_std_timed = deepcopy(nutlin_closed_slice_results_df)

    am_pred_ddg_std_timed["ddg std. (kcal/mol)"] = np.sqrt(am_open_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std_timed["ddg std. (kcal/mol)"] = np.sqrt(nutlin_closed_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)

    # drop dG avg. (kcal/mol) column
    am_pred_ddg_std_timed.drop(columns=["dG avg. (kcal/mol)"], inplace=True)
    nutlin_pred_ddg_std_timed.drop(columns=["dG avg. (kcal/mol)"], inplace=True)

    temporal_am_pred_ddg_std.append(am_pred_ddg_std_timed)
    temporal_nutlin_pred_ddg_std.append(nutlin_pred_ddg_std_timed)


    # am_pred_ddg_std, nutlin_pred_ddg_std

    # rename columns
    am_pred_ddg = am_pred_ddg.rename("ddG avg. (kcal/mol)")
    am_pred_ddg_std = am_pred_ddg_std.rename("ddG std. (kcal/mol)")
    nutlin_pred_ddg = nutlin_pred_ddg.rename("ddG avg. (kcal/mol)")
    nutlin_pred_ddg_std = nutlin_pred_ddg_std.rename("ddG std. (kcal/mol)")

    am_pred_ddg_df = pd.DataFrame(am_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    am_pred_ddg_std = pd.DataFrame(am_pred_ddg_std, columns=["ddG std. (kcal/mol)"])
    nutlin_pred_ddg_df = pd.DataFrame(nutlin_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    nutlin_pred_ddg_std = pd.DataFrame(nutlin_pred_ddg_std, columns=["ddG std. (kcal/mol)"])

    # reindex the dataframes to match the experimental data
    am_pred_ddg_df.reindex(am_exp_ddg_kcal_df.index)
    am_pred_ddg_std.reindex(am_exp_ddg_kcal_df.index)

    print(am_pred_ddg_df, am_pred_ddg_std)

    nutlin_pred_ddg_df.reindex(nutlin_exp_ddg_kcal_df.index)
    nutlin_pred_ddg_std.reindex(nutlin_exp_ddg_kcal_df.index)


    # am_pred_ddg_df, nutlin_pred_ddg_df

    predicted_ddg_df = pd.concat([am_pred_ddg_df, nutlin_pred_ddg_df])
    predicted_ddg_std_df = pd.concat([am_pred_ddg_std, nutlin_pred_ddg_std])

    experimental_ddg_df = pd.concat([am_exp_ddg_kcal_df, nutlin_exp_ddg_kcal_df])

    # predicted_ddg_df, experimental_ddg_df

    # to numpy array
    predicted_ddg = predicted_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    predicted_ddg_std = predicted_ddg_std_df["ddG std. (kcal/mol)"].to_numpy()
    experimental_ddg = experimental_ddg_df["ddG (kcal/mol)"].to_numpy()


    # predicted_ddg, experimental_ddg

    rmse_partial = calculate_rmse(predicted_ddg, experimental_ddg)
    mae_partial = calculate_mae(predicted_ddg, experimental_ddg)

    # rmse_partial, mae_partial

    # convert to numpy arrays for plotting
    am_exp_ddg_kcal_np = am_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    am_pred_ddg_np = am_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    am_pred_ddg_std_np = am_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    nutlin_exp_ddg_kcal_np = nutlin_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_np = nutlin_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_std_np = nutlin_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    temporal_rmse[f"{time_start_ns}-{time_end_ns}"] = rmse_partial
    temporal_mae[f"{time_start_ns}-{time_end_ns}"] = mae_partial
    temporal_std[f"{time_start_ns}-{time_end_ns}"] = predicted_ddg_std.mean()


    ### PLOT OF Dominant Lid States



    fig, axs = plt.subplots(layout='tight')

    x = np.arange(-5, 10, 1)
    y = np.arange(-5, 10, 1)
    # plt.rcParams.update({'font.size': 14})
    ax_x_lim, ax_y_lim = -2, 6
    padding_x = 0.15



    axs.fill_between(x, [i - 1 for i in y], [i + 1 for i in y], alpha=0.2, color='grey')
    axs.fill_between(x, [i - 0.5 for i in y], [i + 0.5 for i in y], alpha=0.2, color='grey')
    axs.errorbar(x=nutlin_exp_ddg_kcal_np, y=nutlin_pred_ddg_np, yerr=nutlin_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#6fc1dcff', ecolor='black', label='Nutlin-3a')
    axs.errorbar(x=am_exp_ddg_kcal_np , y=am_pred_ddg_np, yerr=am_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#F07167', ecolor='black', label='AM-7209')
    axs.legend(loc='upper left')
    axs.set_ylim(ax_x_lim, ax_y_lim)
    axs.set_xlim(ax_x_lim, ax_y_lim)
    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    axs.set_ylabel("Predicted $\Delta \Delta G$ (kcal/mol)")
    axs.set_xlabel("Experimental $\Delta \Delta G$ (kcal/mol)")
    axs.set_box_aspect(1)




    axs.annotate('V14G', (nutlin_exp_ddg_kcal_np[0], nutlin_pred_ddg_np[0]), size=18, xytext=(-55, -8), textcoords="offset pixels")
    axs.annotate('T16G', (nutlin_exp_ddg_kcal_np[1], nutlin_pred_ddg_np[1]), size=18, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('Q18G', (nutlin_exp_ddg_kcal_np[2], nutlin_pred_ddg_np[2]), size=18, xytext=(-55, -5), textcoords="offset pixels")
    axs.annotate('I19G', (nutlin_exp_ddg_kcal_np[3], nutlin_pred_ddg_np[3]), size=18, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (nutlin_exp_ddg_kcal_np[4], nutlin_pred_ddg_np[4]), size=18, xytext=(-100, -5), textcoords="offset pixels")

    axs.annotate('V14G', (am_exp_ddg_kcal_np [0], am_pred_ddg_np[0]), size=18, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G', (am_exp_ddg_kcal_np [1], am_pred_ddg_np[1]), size=18, xytext=(-55, -5), textcoords="offset pixels")
    axs.annotate('Q18G', (am_exp_ddg_kcal_np [2], am_pred_ddg_np[2]), size=18, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('I19G', (am_exp_ddg_kcal_np [3], am_pred_ddg_np[3]), size=18, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (am_exp_ddg_kcal_np [4], am_pred_ddg_np[4]), size=18, xytext=(10, -5), textcoords="offset pixels")

    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    textstr1 = f"RMSE:      {rmse_partial:.2f}     MAE:      {mae_partial:.2f}\n"
    textstr = textstr1 
    # bootstrap statistics
    res_rmse = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='RMSE')
    res_mae = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='MUE')
    res_rho = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='rho')
    res_r2 = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='R2')
    res_ktau = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='KTAU')
    print(res_rmse)

    # append lower and upper bounds to the dictionary
    temporal_rmse_lower[f"{time_start_ns}-{time_end_ns}"] = res_rmse['low']
    temporal_rmse_upper[f"{time_start_ns}-{time_end_ns}"] = res_rmse['high']

    textstr = f"RMSE:        {res_rmse['mle']:.2f} [95%: {res_rmse['low']:.2f}, {res_rmse['high']:.2f}]\nMAE:        {res_mae['mle']:.2f} [95%: {res_mae['low']:.2f}, {res_mae['high']:.2f}]\n rho:        {res_rho['mle']:.2f} [95%: {res_rho['low']:.2f}, {res_rho['high']:.2f}]\n R2:        {res_r2['mle']:.2f} [95%: {res_r2['low']:.2f}, {res_r2['high']:.2f}]\nKTAU:        {res_ktau['mle']:.2f} [95%: {res_ktau['low']:.2f}, {res_ktau['high']:.2f}]"

    # full_title = f"{textstr} {title}"
    full_title = f"{textstr}"
    axs.set_title(full_title, family="monospace", loc='right')
    fig.set_size_inches(10,10)
    #plt.savefig(f'paper_eq_fep_plots/{time_start_ns}-{time_end_ns}ns_ff14sb.pdf', dpi=600, bbox_inches='tight')
    #plt.savefig(f'paper_eq_fep_plots/{time_start_ns}-{time_end_ns}ns_ff14sb.png', dpi=600, bbox_inches='tight')

## NEQ 0 to 100 % (Forward Convergence)

In [ ]:
am_neq_full_results_df = pd.read_csv('neq_fep_analysed_results_0_to_100/am_df_full.csv')
apo_neq_full_results_df = pd.read_csv('neq_fep_analysed_results_0_to_100/apo_df_full.csv')
nutlin_neq_full_results_df = pd.read_csv('neq_fep_analysed_results_0_to_100/nutlin_df_full.csv')

### Plotting

In [ ]:
temporal_rmse = {}
temporal_rmse_lower = {}
temporal_rmse_upper = {}
temporal_mae = {}
temporal_std = {}
temporal_am_pred_ddg_std = []
temporal_nutlin_pred_ddg_std = []


time_start_perc = 0 # ns

for time_end_perc in range(10, 110, 10):

    am_open_slice_results_df = am_neq_full_results_df[(am_neq_full_results_df['time_start_perc'] == 0) & (am_neq_full_results_df['time_end_perc'] == time_end_perc)]
    apo_closed_slice_results_df = apo_neq_full_results_df[(apo_neq_full_results_df['time_start_perc'] == 0) & (apo_neq_full_results_df['time_end_perc'] == time_end_perc)]
    nutlin_closed_slice_results_df = nutlin_neq_full_results_df[(nutlin_neq_full_results_df['time_start_perc'] == 0) & (nutlin_neq_full_results_df['time_end_perc'] == time_end_perc)]

    
    am_pred_ddg = am_open_slice_results_df["dG avg. (kcal/mol)"] - apo_closed_slice_results_df["dG avg. (kcal/mol)"]
    nutlin_pred_ddg = nutlin_closed_slice_results_df["dG avg. (kcal/mol)"] - apo_closed_slice_results_df["dG avg. (kcal/mol)"]

    am_pred_ddg_std = np.sqrt(am_open_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std = np.sqrt(nutlin_closed_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)


    # create separate dataframes for keeping track of time depedenent std and the mutant names
    am_pred_ddg_std_timed = deepcopy(am_open_slice_results_df)
    nutlin_pred_ddg_std_timed = deepcopy(nutlin_closed_slice_results_df)

    am_pred_ddg_std_timed["ddg std. (kcal/mol)"] = np.sqrt(am_open_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std_timed["ddg std. (kcal/mol)"] = np.sqrt(nutlin_closed_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)

    # drop dG avg. (kcal/mol) column
    am_pred_ddg_std_timed.drop(columns=["dG avg. (kcal/mol)"], inplace=True)
    nutlin_pred_ddg_std_timed.drop(columns=["dG avg. (kcal/mol)"], inplace=True)

    temporal_am_pred_ddg_std.append(am_pred_ddg_std_timed)
    temporal_nutlin_pred_ddg_std.append(nutlin_pred_ddg_std_timed)

    # am_pred_ddg_std, nutlin_pred_ddg_std

    # rename columns
    am_pred_ddg = am_pred_ddg.rename("ddG avg. (kcal/mol)")
    am_pred_ddg_std = am_pred_ddg_std.rename("ddG std. (kcal/mol)")
    nutlin_pred_ddg = nutlin_pred_ddg.rename("ddG avg. (kcal/mol)")
    nutlin_pred_ddg_std = nutlin_pred_ddg_std.rename("ddG std. (kcal/mol)")

    am_pred_ddg_df = pd.DataFrame(am_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    am_pred_ddg_std = pd.DataFrame(am_pred_ddg_std, columns=["ddG std. (kcal/mol)"])
    nutlin_pred_ddg_df = pd.DataFrame(nutlin_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    nutlin_pred_ddg_std = pd.DataFrame(nutlin_pred_ddg_std, columns=["ddG std. (kcal/mol)"])

    # reindex the dataframes to match the experimental data
    am_pred_ddg_df.reindex(am_exp_ddg_kcal_df.index)
    am_pred_ddg_std.reindex(am_exp_ddg_kcal_df.index)

    nutlin_pred_ddg_df.reindex(nutlin_exp_ddg_kcal_df.index)
    nutlin_pred_ddg_std.reindex(nutlin_exp_ddg_kcal_df.index)


    # am_pred_ddg_df, nutlin_pred_ddg_df

    predicted_ddg_df = pd.concat([am_pred_ddg_df, nutlin_pred_ddg_df])
    predicted_ddg_std_df = pd.concat([am_pred_ddg_std, nutlin_pred_ddg_std])

    experimental_ddg_df = pd.concat([am_exp_ddg_kcal_df, nutlin_exp_ddg_kcal_df])

    # predicted_ddg_df, experimental_ddg_df

    # to numpy array
    predicted_ddg = predicted_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    predicted_ddg_std = predicted_ddg_std_df["ddG std. (kcal/mol)"].to_numpy()
    experimental_ddg = experimental_ddg_df["ddG (kcal/mol)"].to_numpy()


    # predicted_ddg, experimental_ddg

    rmse_partial = calculate_rmse(predicted_ddg, experimental_ddg)
    mae_partial = calculate_mae(predicted_ddg, experimental_ddg)

    # rmse_partial, mae_partial

    # convert to numpy arrays for plotting
    am_exp_ddg_kcal_np = am_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    am_pred_ddg_np = am_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    am_pred_ddg_std_np = am_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    nutlin_exp_ddg_kcal_np = nutlin_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_np = nutlin_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_std_np = nutlin_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    temporal_rmse[f"{time_start_perc}-{time_end_perc}"] = rmse_partial
    temporal_mae[f"{time_start_perc}-{time_end_perc}"] = mae_partial
    temporal_std[f"{time_start_perc}-{time_end_perc}"] = predicted_ddg_std.mean()


    ### PLOT OF Dominant Lid States



    fig, axs = plt.subplots(layout='tight')

    x = np.arange(-5, 10, 1)
    y = np.arange(-5, 10, 1)
    plt.rcParams.update({'font.size': 14})
    ax_x_lim, ax_y_lim = -2, 6
    padding_x = 0.15



    axs.fill_between(x, [i - 1 for i in y], [i + 1 for i in y], alpha=0.2, color='grey')
    axs.fill_between(x, [i - 0.5 for i in y], [i + 0.5 for i in y], alpha=0.2, color='grey')
    axs.errorbar(x=nutlin_exp_ddg_kcal_np, y=nutlin_pred_ddg_np, yerr=nutlin_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#6fc1dcff', ecolor='black', label='Nutlin-3a')
    axs.errorbar(x=am_exp_ddg_kcal_np , y=am_pred_ddg_np, yerr=am_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#F07167', ecolor='black', label='AM-7209')
    axs.legend(loc='upper left')
    axs.set_ylim(ax_x_lim, ax_y_lim)
    axs.set_xlim(ax_x_lim, ax_y_lim)
    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    axs.set_ylabel("Predicted $\Delta \Delta G$ (kcal/mol)")
    axs.set_xlabel("Experimental $\Delta \Delta G$ (kcal/mol)")
    axs.set_box_aspect(1)



    axs.annotate('V14G', (nutlin_exp_ddg_kcal_np[0], nutlin_pred_ddg_np[0]), size=12, xytext=(-45, -5), textcoords="offset pixels")
    axs.annotate('T16G', (nutlin_exp_ddg_kcal_np[1], nutlin_pred_ddg_np[1]), size=12, xytext=(15, -6), textcoords="offset pixels")
    axs.annotate('Q18G', (nutlin_exp_ddg_kcal_np[2], nutlin_pred_ddg_np[2]), size=12, xytext=(-45, -5), textcoords="offset pixels")
    axs.annotate('I19G', (nutlin_exp_ddg_kcal_np[3], nutlin_pred_ddg_np[3]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (nutlin_exp_ddg_kcal_np[4], nutlin_pred_ddg_np[4]), size=12, xytext=(-80, -5), textcoords="offset pixels")

    axs.annotate('V14G', (am_exp_ddg_kcal_np [0], am_pred_ddg_np[0]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G', (am_exp_ddg_kcal_np [1], am_pred_ddg_np[1]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('Q18G', (am_exp_ddg_kcal_np [2], am_pred_ddg_np[2]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('I19G', (am_exp_ddg_kcal_np [3], am_pred_ddg_np[3]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (am_exp_ddg_kcal_np [4], am_pred_ddg_np[4]), size=12, xytext=(15, -5), textcoords="offset pixels")

    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    # from matplotlib.ticker import FormatStrFormatter

    # axs.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    # axs.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    textstr1 = f"RMSE:      {rmse_partial:.2f}     MAE:      {mae_partial:.2f}\n"
    textstr = textstr1 
    #title = r'MDM2 Dominant Lid States Only (BSS Inputs - Mixture of Replicas)'
    title = f'MDM2 NEQ. FEP Dominant State Lid Mutations (N = {len(experimental_ddg)})'
    # bootstrap statistics
    res_rmse = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='RMSE')
    print(res_rmse)
    res_mae = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='MUE')
    res_rho = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='rho')

    # append lower and upper bounds to the dictionary
    temporal_rmse_lower[f"{time_start_perc}-{time_end_perc}"] = res_rmse['low']
    temporal_rmse_upper[f"{time_start_perc}-{time_end_perc}"] = res_rmse['high']


    textstr = f"RMSE:        {res_rmse['mle']:.2f} [95%: {res_rmse['low']:.2f}, {res_rmse['high']:.2f}]\nMAE:        {res_mae['mle']:.2f} [95%: {res_mae['low']:.2f}, {res_mae['high']:.2f}]\n"
    full_title = f"{textstr} {title}"
    axs.set_title(full_title, family="monospace", loc='right')
    fig.set_size_inches(10,10)
    # plt.savefig(f'paper_neq_fep_plots/{time_start_perc}-{time_end_perc}_perc.pdf', dpi=500, bbox_inches='tight')

## NEQ 100 to 10 % (Forward Discard)

In [ ]:
am_neq_full_results_df = pd.read_csv('neq_fep_analysed_results_100_to_10/am_df_full.csv')
apo_neq_full_results_df = pd.read_csv('neq_fep_analysed_results_100_to_10/apo_df_full.csv')
nutlin_neq_full_results_df = pd.read_csv('neq_fep_analysed_results_100_to_10/nutlin_df_full.csv')

### Plotting

In [ ]:
temporal_rmse = {}
temporal_rmse_lower = {}
temporal_rmse_upper = {}
temporal_mae = {}
temporal_std = {}
temporal_am_pred_ddg_std = []
temporal_nutlin_pred_ddg_std = []

time_end_perc = 100

for time_start_perc in range(10, 100, 10):


    am_open_slice_results_df = am_neq_full_results_df[(am_neq_full_results_df['time_start_perc'] == time_start_perc) & (am_neq_full_results_df['time_end_perc'] == 100)]
    apo_closed_slice_results_df = apo_neq_full_results_df[(apo_neq_full_results_df['time_start_perc'] == time_start_perc) & (apo_neq_full_results_df['time_end_perc'] == 100)]
    nutlin_closed_slice_results_df = nutlin_neq_full_results_df[(nutlin_neq_full_results_df['time_start_perc'] == time_start_perc) & (nutlin_neq_full_results_df['time_end_perc'] == 100)]

    
    am_pred_ddg = am_open_slice_results_df["dG avg. (kcal/mol)"] - apo_closed_slice_results_df["dG avg. (kcal/mol)"]
    nutlin_pred_ddg = nutlin_closed_slice_results_df["dG avg. (kcal/mol)"] - apo_closed_slice_results_df["dG avg. (kcal/mol)"]

    am_pred_ddg_std = np.sqrt(am_open_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std = np.sqrt(nutlin_closed_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)

    # create separate dataframes for keeping track of time depedenent std and the mutant names
    am_pred_ddg_std_timed = deepcopy(am_open_slice_results_df)
    nutlin_pred_ddg_std_timed = deepcopy(nutlin_closed_slice_results_df)

    am_pred_ddg_std_timed["ddg std. (kcal/mol)"] = np.sqrt(am_open_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std_timed["ddg std. (kcal/mol)"] = np.sqrt(nutlin_closed_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)

    # drop dG avg. (kcal/mol) column
    am_pred_ddg_std_timed.drop(columns=["dG avg. (kcal/mol)"], inplace=True)
    nutlin_pred_ddg_std_timed.drop(columns=["dG avg. (kcal/mol)"], inplace=True)

    temporal_am_pred_ddg_std.append(am_pred_ddg_std_timed)
    temporal_nutlin_pred_ddg_std.append(nutlin_pred_ddg_std_timed)

    # am_pred_ddg_std, nutlin_pred_ddg_std

    # rename columns
    am_pred_ddg = am_pred_ddg.rename("ddG avg. (kcal/mol)")
    am_pred_ddg_std = am_pred_ddg_std.rename("ddG std. (kcal/mol)")
    nutlin_pred_ddg = nutlin_pred_ddg.rename("ddG avg. (kcal/mol)")
    nutlin_pred_ddg_std = nutlin_pred_ddg_std.rename("ddG std. (kcal/mol)")

    am_pred_ddg_df = pd.DataFrame(am_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    am_pred_ddg_std = pd.DataFrame(am_pred_ddg_std, columns=["ddG std. (kcal/mol)"])
    nutlin_pred_ddg_df = pd.DataFrame(nutlin_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    nutlin_pred_ddg_std = pd.DataFrame(nutlin_pred_ddg_std, columns=["ddG std. (kcal/mol)"])

    # reindex the dataframes to match the experimental data
    am_pred_ddg_df.reindex(am_exp_ddg_kcal_df.index)
    am_pred_ddg_std.reindex(am_exp_ddg_kcal_df.index)

    nutlin_pred_ddg_df.reindex(nutlin_exp_ddg_kcal_df.index)
    nutlin_pred_ddg_std.reindex(nutlin_exp_ddg_kcal_df.index)


    # am_pred_ddg_df, nutlin_pred_ddg_df

    predicted_ddg_df = pd.concat([am_pred_ddg_df, nutlin_pred_ddg_df])
    predicted_ddg_std_df = pd.concat([am_pred_ddg_std, nutlin_pred_ddg_std])

    experimental_ddg_df = pd.concat([am_exp_ddg_kcal_df, nutlin_exp_ddg_kcal_df])

    # predicted_ddg_df, experimental_ddg_df

    # to numpy array
    predicted_ddg = predicted_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    predicted_ddg_std = predicted_ddg_std_df["ddG std. (kcal/mol)"].to_numpy()
    experimental_ddg = experimental_ddg_df["ddG (kcal/mol)"].to_numpy()


    # predicted_ddg, experimental_ddg

    rmse_partial = calculate_rmse(predicted_ddg, experimental_ddg)
    mae_partial = calculate_mae(predicted_ddg, experimental_ddg)

    # rmse_partial, mae_partial

    # convert to numpy arrays for plotting
    am_exp_ddg_kcal_np = am_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    am_pred_ddg_np = am_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    am_pred_ddg_std_np = am_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    nutlin_exp_ddg_kcal_np = nutlin_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_np = nutlin_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_std_np = nutlin_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    temporal_rmse[f"{time_start_perc}-{time_end_perc}"] = rmse_partial
    temporal_mae[f"{time_start_perc}-{time_end_perc}"] = mae_partial
    temporal_std[f"{time_start_perc}-{time_end_perc}"] = predicted_ddg_std.mean()


    ### PLOT OF Dominant Lid States



    fig, axs = plt.subplots(layout='tight')

    x = np.arange(-5, 10, 1)
    y = np.arange(-5, 10, 1)
    plt.rcParams.update({'font.size': 14})
    ax_x_lim, ax_y_lim = -2, 6
    padding_x = 0.15



    axs.fill_between(x, [i - 1 for i in y], [i + 1 for i in y], alpha=0.2, color='grey')
    axs.fill_between(x, [i - 0.5 for i in y], [i + 0.5 for i in y], alpha=0.2, color='grey')
    axs.errorbar(x=nutlin_exp_ddg_kcal_np, y=nutlin_pred_ddg_np, yerr=nutlin_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#6fc1dcff', ecolor='black', label='Nutlin-3a')
    axs.errorbar(x=am_exp_ddg_kcal_np , y=am_pred_ddg_np, yerr=am_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#F07167', ecolor='black', label='AM-7209')
    axs.legend(loc='upper left')
    axs.set_ylim(ax_x_lim, ax_y_lim)
    axs.set_xlim(ax_x_lim, ax_y_lim)
    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    axs.set_ylabel("Predicted $\Delta \Delta G$ (kcal/mol)")
    axs.set_xlabel("Experimental $\Delta \Delta G$ (kcal/mol)")
    axs.set_box_aspect(1)




    axs.annotate('V14G', (nutlin_exp_ddg_kcal_np[0], nutlin_pred_ddg_np[0]), size=12, xytext=(-45, -5), textcoords="offset pixels")
    axs.annotate('T16G', (nutlin_exp_ddg_kcal_np[1], nutlin_pred_ddg_np[1]), size=12, xytext=(15, -6), textcoords="offset pixels")
    axs.annotate('Q18G', (nutlin_exp_ddg_kcal_np[2], nutlin_pred_ddg_np[2]), size=12, xytext=(-45, -5), textcoords="offset pixels")
    axs.annotate('I19G', (nutlin_exp_ddg_kcal_np[3], nutlin_pred_ddg_np[3]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (nutlin_exp_ddg_kcal_np[4], nutlin_pred_ddg_np[4]), size=12, xytext=(-80, -5), textcoords="offset pixels")

    axs.annotate('V14G', (am_exp_ddg_kcal_np [0], am_pred_ddg_np[0]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G', (am_exp_ddg_kcal_np [1], am_pred_ddg_np[1]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('Q18G', (am_exp_ddg_kcal_np [2], am_pred_ddg_np[2]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('I19G', (am_exp_ddg_kcal_np [3], am_pred_ddg_np[3]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (am_exp_ddg_kcal_np [4], am_pred_ddg_np[4]), size=12, xytext=(15, -5), textcoords="offset pixels")

    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    # from matplotlib.ticker import FormatStrFormatter

    # axs.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    # axs.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    textstr1 = f"RMSE:      {rmse_partial:.2f}     MAE:      {mae_partial:.2f}\n"
    textstr = textstr1 
    #title = r'MDM2 Dominant Lid States Only (BSS Inputs - Mixture of Replicas)'
    title = f'MDM2 NEQ. FEP Dominant State Lid Mutations (N = {len(experimental_ddg)})'
    # bootstrap statistics
    res_rmse = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='RMSE')
    print(res_rmse)
    res_mae = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='MUE')
    res_rho = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='rho')

    # append lower and upper bounds to the dictionary
    temporal_rmse_lower[f"{time_start_perc}-{time_end_perc}"] = res_rmse['low']
    temporal_rmse_upper[f"{time_start_perc}-{time_end_perc}"] = res_rmse['high']


    textstr = f"RMSE:        {res_rmse['mle']:.2f} [95%: {res_rmse['low']:.2f}, {res_rmse['high']:.2f}]\nMAE:        {res_mae['mle']:.2f} [95%: {res_mae['low']:.2f}, {res_mae['high']:.2f}]\n"
    full_title = f"{textstr} {title}"
    axs.set_title(full_title, family="monospace", loc='right')
    fig.set_size_inches(10,10)
    # plt.savefig(f'paper_neq_fep_plots/{time_start_perc}-{time_end_perc}_perc.pdf', dpi=500, bbox_inches='tight')

## NEQ Pooled 0 to 100 % (Forward Convergence)

In [ ]:
mutants = ["v14g", "t16g", "q18g", "i19g", "t16g_i19g"]
am_neq_full_results_pooled_df = pd.DataFrame()
apo_neq_full_results_pooled_df = pd.DataFrame()
nutlin_neq_full_results_pooled_df = pd.DataFrame()

for mutant in mutants:

    time_start = 0
    time_end_slice = [275, 550, 825, 1100, 1375, 1650, 1925, 2200, 2475, 2750]

    for time_end in time_end_slice:

        am_open_pooled_df = pd.read_csv(f"neq_xvg_analysis/am_open/system_pooled_results/results_0_{time_end}.dat",
                                names=["mutant", "dG (kcal/mol)"])

        # Define the string to search for
        search_string = rf'\b{mutant}\b'

        # Apply the string containment check across all elements
        contains_string = am_open_pooled_df.applymap(lambda x: specific_pattern_match(x, search_string))

        # Filter the DataFrame to get only the rows where any element contains the string
        filtered_pooled_df = am_open_pooled_df[contains_string.any(axis=1)]

        # Use a regular expression to extract floating point values
        # The regex pattern r'([-+]?\d*\.\d+|\d+)' matches both integers and floating point numbers
        filtered_pooled_df['extracted dG (kcal/mol)'] = filtered_pooled_df["dG (kcal/mol)"].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)

        if len(filtered_pooled_df['extracted dG (kcal/mol)']) != 1 or filtered_pooled_df['extracted dG (kcal/mol)'].isnull().sum() > 0:
            raise ValueError("The number of extracted values is not equal to 1")

        # Display the DataFrame with the extracted values
        # mean_dg = filtered_pooled_df['extracted dG (kcal/mol)'].mean()
        # std_dg = filtered_pooled_df['extracted dG (kcal/mol)'].std()
        pooled_dg = filtered_pooled_df['extracted dG (kcal/mol)'].tolist()
        time_end_perc = time_end / time_end_slice[-1] * 100
        processed_pooled_df = pd.DataFrame.from_dict({"mutant": mutant, "dG pooled (kcal/mol)": pooled_dg[0], "time_start_perc": time_start, "time_end_perc": time_end_perc}, orient="index").T
        am_neq_full_results_pooled_df = pd.concat([am_neq_full_results_pooled_df, processed_pooled_df])
        am_neq_full_results_pooled_df.reset_index(drop=True, inplace=True)


for mutant in mutants:

    time_start = 0
    # if mutant == "t16g_i19g":
    #     time_end_slice= [525, 1050, 1575, 2100, 2625, 3150, 3675, 4200, 4725, 5250]
    # else:
    time_end_slice = [275, 550, 825, 1100, 1375, 1650, 1925, 2200, 2475, 2750]


    for time_end in time_end_slice:

        apo_closed_pooled_df = pd.read_csv(f"neq_xvg_analysis/apo_closed/system_pooled_results/results_0_{time_end}.dat",
                                names=["mutant", "dG (kcal/mol)"])

        # Define the string to search for
        search_string = rf'\b{mutant}\b'

        # Apply the string containment check across all elements
        contains_string = apo_closed_pooled_df.applymap(lambda x: specific_pattern_match(x, search_string))

        # Filter the DataFrame to get only the rows where any element contains the string
        filtered_pooled_df = apo_closed_pooled_df[contains_string.any(axis=1)]

        # Use a regular expression to extract floating point values
        # The regex pattern r'([-+]?\d*\.\d+|\d+)' matches both integers and floating point numbers
        filtered_pooled_df['extracted dG (kcal/mol)'] = filtered_pooled_df["dG (kcal/mol)"].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)

        if len(filtered_pooled_df['extracted dG (kcal/mol)']) != 1 or filtered_pooled_df['extracted dG (kcal/mol)'].isnull().sum() > 0:
            raise ValueError("The number of extracted values is not equal to 1")

        # Display the DataFrame with the extracted values
        # mean_dg = filtered_pooled_df['extracted dG (kcal/mol)'].mean()
        # std_dg = filtered_pooled_df['extracted dG (kcal/mol)'].std()
        time_end_perc = time_end / time_end_slice[-1] * 100
        pooled_dg = filtered_pooled_df['extracted dG (kcal/mol)'].tolist()
        processed_pooled_df = pd.DataFrame.from_dict({"mutant": mutant, "dG pooled (kcal/mol)": pooled_dg[0], "time_start_perc": time_start, "time_end_perc": time_end_perc}, orient="index").T
        apo_neq_full_results_pooled_df = pd.concat([apo_neq_full_results_pooled_df, processed_pooled_df])
        apo_neq_full_results_pooled_df.reset_index(drop=True, inplace=True)


for mutant in mutants:

    time_start = 0
    time_end_slice = [275, 550, 825, 1100, 1375, 1650, 1925, 2200, 2475, 2750]

    for time_end in time_end_slice:

        nutlin_closed_pooled_df = pd.read_csv(f"neq_xvg_analysis/nutlin_closed/system_pooled_results/results_0_{time_end}.dat",
                                names=["mutant", "dG (kcal/mol)"])

        # Define the string to search for
        search_string = rf'\b{mutant}\b'

        # Apply the string containment check across all elements
        contains_string = nutlin_closed_pooled_df.applymap(lambda x: specific_pattern_match(x, search_string))

        # Filter the DataFrame to get only the rows where any element contains the string
        filtered_pooled_df = nutlin_closed_pooled_df[contains_string.any(axis=1)]

        # Use a regular expression to extract floating point values
        # The regex pattern r'([-+]?\d*\.\d+|\d+)' matches both integers and floating point numbers
        filtered_pooled_df['extracted dG (kcal/mol)'] = filtered_pooled_df["dG (kcal/mol)"].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)

        if len(filtered_pooled_df['extracted dG (kcal/mol)']) != 1 or filtered_pooled_df['extracted dG (kcal/mol)'].isnull().sum() > 0:
            raise ValueError("The number of extracted values is not equal to 1")

        # Display the DataFrame with the extracted values
        # mean_dg = filtered_pooled_df['extracted dG (kcal/mol)'].mean()
        # std_dg = filtered_pooled_df['extracted dG (kcal/mol)'].std()
        time_end_perc = time_end / time_end_slice[-1] * 100
        pooled_dg = filtered_pooled_df['extracted dG (kcal/mol)'].tolist()
        processed_pooled_df = pd.DataFrame.from_dict({"mutant": mutant, "dG pooled (kcal/mol)": pooled_dg[0], "time_start_perc": time_start, "time_end_perc": time_end_perc}, orient="index").T
        nutlin_neq_full_results_pooled_df = pd.concat([nutlin_neq_full_results_pooled_df, processed_pooled_df])
        nutlin_neq_full_results_pooled_df.reset_index(drop=True, inplace=True)


# convert dtypes to float
am_neq_full_results_pooled_df["dG pooled (kcal/mol)"] = am_neq_full_results_pooled_df["dG pooled (kcal/mol)"].astype(float)

apo_neq_full_results_pooled_df["dG pooled (kcal/mol)"] = apo_neq_full_results_pooled_df["dG pooled (kcal/mol)"].astype(float)

nutlin_neq_full_results_pooled_df["dG pooled (kcal/mol)"] = nutlin_neq_full_results_pooled_df["dG pooled (kcal/mol)"].astype(float)

am_neq_full_results_pooled_df

In [ ]:
am_neq_full_results_pooled_df["dg std. (kcal/mol)"] = am_neq_full_results_df["dg std. (kcal/mol)"]
apo_neq_full_results_pooled_df["dg std. (kcal/mol)"] = apo_neq_full_results_df["dg std. (kcal/mol)"]
nutlin_neq_full_results_pooled_df["dg std. (kcal/mol)"] = nutlin_neq_full_results_df["dg std. (kcal/mol)"]

In [ ]:
nutlin_neq_full_results_df

### Plotting

In [ ]:
temporal_rmse = {}
temporal_rmse_lower = {}
temporal_rmse_upper = {}
temporal_mae = {}


time_start_perc = 0 # ns

for time_end_perc in range(10, 110, 10):

    am_open_slice_results_pooled_df = am_neq_full_results_pooled_df[(am_neq_full_results_pooled_df['time_start_perc'] == 0) & (am_neq_full_results_pooled_df['time_end_perc'] == time_end_perc)]
    apo_closed_slice_results_pooled_df = apo_neq_full_results_pooled_df[(apo_neq_full_results_pooled_df['time_start_perc'] == 0) & (apo_neq_full_results_pooled_df['time_end_perc'] == time_end_perc)]
    nutlin_closed_slice_results_pooled_df = nutlin_neq_full_results_pooled_df[(nutlin_neq_full_results_pooled_df['time_start_perc'] == 0) & (nutlin_neq_full_results_pooled_df['time_end_perc'] == time_end_perc)]

    
    am_pred_ddg = am_open_slice_results_pooled_df["dG pooled (kcal/mol)"] - apo_closed_slice_results_pooled_df["dG pooled (kcal/mol)"]
    nutlin_pred_ddg = nutlin_closed_slice_results_pooled_df["dG pooled (kcal/mol)"] - apo_closed_slice_results_pooled_df["dG pooled (kcal/mol)"]

    am_pred_ddg_std = np.sqrt(am_open_slice_results_pooled_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_pooled_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std = np.sqrt(nutlin_closed_slice_results_pooled_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_pooled_df["dg std. (kcal/mol)"]**2)

    # am_pred_ddg_std, nutlin_pred_ddg_std

    # rename columns
    am_pred_ddg = am_pred_ddg.rename("ddG avg. (kcal/mol)")
    am_pred_ddg_std = am_pred_ddg_std.rename("ddG std. (kcal/mol)")
    nutlin_pred_ddg = nutlin_pred_ddg.rename("ddG avg. (kcal/mol)")
    nutlin_pred_ddg_std = nutlin_pred_ddg_std.rename("ddG std. (kcal/mol)")

    am_pred_ddg_pooled_df = pd.DataFrame(am_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    am_pred_ddg_std = pd.DataFrame(am_pred_ddg_std, columns=["ddG std. (kcal/mol)"])
    nutlin_pred_ddg_pooled_df = pd.DataFrame(nutlin_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    nutlin_pred_ddg_std = pd.DataFrame(nutlin_pred_ddg_std, columns=["ddG std. (kcal/mol)"])

    # reindex the dataframes to match the experimental data
    am_pred_ddg_pooled_df.reindex(am_exp_ddg_kcal_df.index)
    am_pred_ddg_std.reindex(am_exp_ddg_kcal_df.index)

    nutlin_pred_ddg_pooled_df.reindex(nutlin_exp_ddg_kcal_df.index)
    nutlin_pred_ddg_std.reindex(nutlin_exp_ddg_kcal_df.index)


    # am_pred_ddg_pooled_df, nutlin_pred_ddg_pooled_df

    predicted_ddg_pooled_df = pd.concat([am_pred_ddg_pooled_df, nutlin_pred_ddg_pooled_df])
    predicted_ddg_std_pooled_df = pd.concat([am_pred_ddg_std, nutlin_pred_ddg_std])

    experimental_ddg_pooled_df = pd.concat([am_exp_ddg_kcal_df, nutlin_exp_ddg_kcal_df])

    # predicted_ddg_pooled_df, experimental_ddg_pooled_df

    # to numpy array
    predicted_ddg = predicted_ddg_pooled_df["ddG avg. (kcal/mol)"].to_numpy()
    predicted_ddg_std = predicted_ddg_std_pooled_df["ddG std. (kcal/mol)"].to_numpy()
    experimental_ddg = experimental_ddg_pooled_df["ddG (kcal/mol)"].to_numpy()


    # predicted_ddg, experimental_ddg

    rmse_partial = calculate_rmse(predicted_ddg, experimental_ddg)
    mae_partial = calculate_mae(predicted_ddg, experimental_ddg)

    # rmse_partial, mae_partial

    # convert to numpy arrays for plotting
    am_exp_ddg_kcal_np = am_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    am_pred_ddg_np = am_pred_ddg_pooled_df["ddG avg. (kcal/mol)"].to_numpy()
    am_pred_ddg_std_np = am_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    nutlin_exp_ddg_kcal_np = nutlin_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_np = nutlin_pred_ddg_pooled_df["ddG avg. (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_std_np = nutlin_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    temporal_rmse[f"{time_start_perc}-{time_end_perc}"] = rmse_partial
    temporal_mae[f"{time_start_perc}-{time_end_perc}"] = mae_partial
    # temporal_std[f"{time_start_perc}-{time_end_perc}"] = predicted_ddg_std.mean()


    ### PLOT OF Dominant Lid States



    fig, axs = plt.subplots(layout='tight')

    x = np.arange(-5, 10, 1)
    y = np.arange(-5, 10, 1)
    plt.rcParams.update({'font.size': 14})
    ax_x_lim, ax_y_lim = -2, 6
    padding_x = 0.15



    axs.fill_between(x, [i - 1 for i in y], [i + 1 for i in y], alpha=0.2, color='grey')
    axs.fill_between(x, [i - 0.5 for i in y], [i + 0.5 for i in y], alpha=0.2, color='grey')
    axs.errorbar(x=nutlin_exp_ddg_kcal_np, y=nutlin_pred_ddg_np, yerr=nutlin_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#6fc1dcff', ecolor='black', label='Nutlin-3a')
    axs.errorbar(x=am_exp_ddg_kcal_np , y=am_pred_ddg_np, yerr=am_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#F07167', ecolor='black', label='AM-7209')
    axs.legend(loc='upper left')
    axs.set_ylim(ax_x_lim, ax_y_lim)
    axs.set_xlim(ax_x_lim, ax_y_lim)
    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    axs.set_ylabel("Predicted $\Delta \Delta G$ (kcal/mol)")
    axs.set_xlabel("Experimental $\Delta \Delta G$ (kcal/mol)")
    axs.set_box_aspect(1)




    axs.annotate('V14G', (nutlin_exp_ddg_kcal_np[0], nutlin_pred_ddg_np[0]), size=12, xytext=(-45, -5), textcoords="offset pixels")
    axs.annotate('T16G', (nutlin_exp_ddg_kcal_np[1], nutlin_pred_ddg_np[1]), size=12, xytext=(15, -6), textcoords="offset pixels")
    axs.annotate('Q18G', (nutlin_exp_ddg_kcal_np[2], nutlin_pred_ddg_np[2]), size=12, xytext=(-45, -5), textcoords="offset pixels")
    axs.annotate('I19G', (nutlin_exp_ddg_kcal_np[3], nutlin_pred_ddg_np[3]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (nutlin_exp_ddg_kcal_np[4], nutlin_pred_ddg_np[4]), size=12, xytext=(-80, -5), textcoords="offset pixels")

    axs.annotate('V14G', (am_exp_ddg_kcal_np [0], am_pred_ddg_np[0]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G', (am_exp_ddg_kcal_np [1], am_pred_ddg_np[1]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('Q18G', (am_exp_ddg_kcal_np [2], am_pred_ddg_np[2]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('I19G', (am_exp_ddg_kcal_np [3], am_pred_ddg_np[3]), size=12, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (am_exp_ddg_kcal_np [4], am_pred_ddg_np[4]), size=12, xytext=(15, -5), textcoords="offset pixels")

    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    # from matplotlib.ticker import FormatStrFormatter

    # axs.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    # axs.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    textstr1 = f"RMSE:      {rmse_partial:.2f}     MAE:      {mae_partial:.2f}\n"
    textstr = textstr1 
    #title = r'MDM2 Dominant Lid States Only (BSS Inputs - Mixture of Replicas)'
    title = f'MDM2 NEQ. FEP Dominant State Lid Mutations (N = {len(experimental_ddg)})'
    # bootstrap statistics
    res_rmse = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='RMSE')
    print(res_rmse)
    res_mae = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='MUE')
    res_rho = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='rho')

    # append lower and upper bounds to the dictionary
    temporal_rmse_lower[f"{time_start_perc}-{time_end_perc}"] = res_rmse['low']
    temporal_rmse_upper[f"{time_start_perc}-{time_end_perc}"] = res_rmse['high']


    textstr = f"RMSE:        {res_rmse['mle']:.2f} [95%: {res_rmse['low']:.2f}, {res_rmse['high']:.2f}]\nMAE:        {res_mae['mle']:.2f} [95%: {res_mae['low']:.2f}, {res_mae['high']:.2f}]\n"
    full_title = f"{textstr} {title}"
    axs.set_title(full_title, family="monospace", loc='right')
    fig.set_size_inches(10,10)
    # plt.savefig(f'paper_neq_fep_plots/{time_start_perc}-{time_end_perc}_perc_pooled.pdf', dpi=500, bbox_inches='tight')

## NEQ Pooled 100 to 10 % (Forward Discard)

In [ ]:
mutants = ["v14g", "t16g", "q18g", "i19g", "t16g_i19g"]
am_neq_full_results_pooled_df = pd.DataFrame()
apo_neq_full_results_pooled_df = pd.DataFrame()
nutlin_neq_full_results_pooled_df = pd.DataFrame()

time_end = 2750

for mutant in mutants:


    time_start_slice = [275, 550, 825, 1100, 1375, 1650, 1925, 2200, 2475]
    
    for time_start in time_start_slice:

        am_open_pooled_df = pd.read_csv(f"neq_xvg_analysis/am_open/system_pooled_results/results_{time_start}_to_{time_end}.dat",
                                names=["mutant", "dG (kcal/mol)"])

        # Define the string to search for
        search_string = rf'\b{mutant}\b'

        # Apply the string containment check across all elements
        contains_string = am_open_pooled_df.applymap(lambda x: specific_pattern_match(x, search_string))

        # Filter the DataFrame to get only the rows where any element contains the string
        filtered_pooled_df = am_open_pooled_df[contains_string.any(axis=1)]

        # Use a regular expression to extract floating point values
        # The regex pattern r'([-+]?\d*\.\d+|\d+)' matches both integers and floating point numbers
        filtered_pooled_df['extracted dG (kcal/mol)'] = filtered_pooled_df["dG (kcal/mol)"].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)

        if len(filtered_pooled_df['extracted dG (kcal/mol)']) != 1 or filtered_pooled_df['extracted dG (kcal/mol)'].isnull().sum() > 0:
            raise ValueError("The number of extracted values is not equal to 1")

        # Display the DataFrame with the extracted values
        # mean_dg = filtered_pooled_df['extracted dG (kcal/mol)'].mean()
        # std_dg = filtered_pooled_df['extracted dG (kcal/mol)'].std()
        pooled_dg = filtered_pooled_df['extracted dG (kcal/mol)'].tolist()
        time_start_perc = time_start / time_end * 100
        time_end_perc = 100
        processed_pooled_df = pd.DataFrame.from_dict({"mutant": mutant, "dG pooled (kcal/mol)": pooled_dg[0], "time_start_perc": time_start_perc, "time_end_perc": time_end_perc}, orient="index").T
        am_neq_full_results_pooled_df = pd.concat([am_neq_full_results_pooled_df, processed_pooled_df])
        am_neq_full_results_pooled_df.reset_index(drop=True, inplace=True)


for mutant in mutants:

    time_start_slice = [275, 550, 825, 1100, 1375, 1650, 1925, 2200, 2475]

    for time_start in time_start_slice:

        apo_closed_pooled_df = pd.read_csv(f"neq_xvg_analysis/apo_closed/system_pooled_results/results_{time_start}_to_{time_end}.dat",
                                names=["mutant", "dG (kcal/mol)"])

        # Define the string to search for
        search_string = rf'\b{mutant}\b'

        # Apply the string containment check across all elements
        contains_string = apo_closed_pooled_df.applymap(lambda x: specific_pattern_match(x, search_string))

        # Filter the DataFrame to get only the rows where any element contains the string
        filtered_pooled_df = apo_closed_pooled_df[contains_string.any(axis=1)]

        # Use a regular expression to extract floating point values
        # The regex pattern r'([-+]?\d*\.\d+|\d+)' matches both integers and floating point numbers
        filtered_pooled_df['extracted dG (kcal/mol)'] = filtered_pooled_df["dG (kcal/mol)"].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)

        if len(filtered_pooled_df['extracted dG (kcal/mol)']) != 1 or filtered_pooled_df['extracted dG (kcal/mol)'].isnull().sum() > 0:
            raise ValueError("The number of extracted values is not equal to 1")

        # Display the DataFrame with the extracted values
        # mean_dg = filtered_pooled_df['extracted dG (kcal/mol)'].mean()
        # std_dg = filtered_pooled_df['extracted dG (kcal/mol)'].std()
        pooled_dg = filtered_pooled_df['extracted dG (kcal/mol)'].tolist()
        time_start_perc = time_start / time_end * 100
        time_end_perc = 100
        processed_pooled_df = pd.DataFrame.from_dict({"mutant": mutant, "dG pooled (kcal/mol)": pooled_dg[0], "time_start_perc": time_start_perc, "time_end_perc": time_end_perc}, orient="index").T
        apo_neq_full_results_pooled_df = pd.concat([apo_neq_full_results_pooled_df, processed_pooled_df])
        apo_neq_full_results_pooled_df.reset_index(drop=True, inplace=True)


for mutant in mutants:

    # if mutant == "t16g_i19g":
    #     time_end = 5250
    #     time_start_slice= [525, 1050, 1575, 2100, 2625, 3150, 3675, 4200, 4725]
    # else:
    #     time_end = 2750
    #     time_start_slice = [275, 550, 825, 1100, 1375, 1650, 1925, 2200, 2475]

    time_start_slice = [275, 550, 825, 1100, 1375, 1650, 1925, 2200, 2475]

    for time_start in time_start_slice:

        nutlin_closed_pooled_df = pd.read_csv(f"neq_xvg_analysis/nutlin_closed/system_pooled_results/results_{time_start}_to_{time_end}.dat",
                                names=["mutant", "dG (kcal/mol)"])

        # Define the string to search for
        search_string = rf'\b{mutant}\b'

        # Apply the string containment check across all elements
        contains_string = nutlin_closed_pooled_df.applymap(lambda x: specific_pattern_match(x, search_string))

        # Filter the DataFrame to get only the rows where any element contains the string
        filtered_pooled_df = nutlin_closed_pooled_df[contains_string.any(axis=1)]

        # Use a regular expression to extract floating point values
        # The regex pattern r'([-+]?\d*\.\d+|\d+)' matches both integers and floating point numbers
        filtered_pooled_df['extracted dG (kcal/mol)'] = filtered_pooled_df["dG (kcal/mol)"].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)

        if len(filtered_pooled_df['extracted dG (kcal/mol)']) != 1 or filtered_pooled_df['extracted dG (kcal/mol)'].isnull().sum() > 0:
            raise ValueError("The number of extracted values is not equal to 1")

        # Display the DataFrame with the extracted values
        # mean_dg = filtered_pooled_df['extracted dG (kcal/mol)'].mean()
        # std_dg = filtered_pooled_df['extracted dG (kcal/mol)'].std()
        pooled_dg = filtered_pooled_df['extracted dG (kcal/mol)'].tolist()
        time_start_perc = time_start / time_end * 100
        time_end_perc = 100
        processed_pooled_df = pd.DataFrame.from_dict({"mutant": mutant, "dG pooled (kcal/mol)": pooled_dg[0], "time_start_perc": time_start_perc, "time_end_perc": time_end_perc}, orient="index").T
        nutlin_neq_full_results_pooled_df = pd.concat([nutlin_neq_full_results_pooled_df, processed_pooled_df])
        nutlin_neq_full_results_pooled_df.reset_index(drop=True, inplace=True)


# convert dtypes to float
am_neq_full_results_pooled_df["dG pooled (kcal/mol)"] = am_neq_full_results_pooled_df["dG pooled (kcal/mol)"].astype(float)

apo_neq_full_results_pooled_df["dG pooled (kcal/mol)"] = apo_neq_full_results_pooled_df["dG pooled (kcal/mol)"].astype(float)

nutlin_neq_full_results_pooled_df["dG pooled (kcal/mol)"] = nutlin_neq_full_results_pooled_df["dG pooled (kcal/mol)"].astype(float)

am_neq_full_results_pooled_df

In [ ]:
# Use non-pooled data to get the standard deviation
am_neq_full_results_pooled_df["dg std. (kcal/mol)"] = am_neq_full_results_df["dg std. (kcal/mol)"]
apo_neq_full_results_pooled_df["dg std. (kcal/mol)"] = apo_neq_full_results_df["dg std. (kcal/mol)"]
nutlin_neq_full_results_pooled_df["dg std. (kcal/mol)"] = nutlin_neq_full_results_df["dg std. (kcal/mol)"]

### Plotting

In [ ]:
sns.set_theme(style="ticks")
sns.set_context("paper", font_scale=2)

temporal_rmse = {}
temporal_rmse_lower = {}
temporal_rmse_upper = {}
temporal_mae = {}
# temporal_std = {}

time_end_perc = 100

for time_start_perc in range(10, 100, 10):


    am_open_slice_results_pooled_df = am_neq_full_results_pooled_df[(am_neq_full_results_pooled_df['time_start_perc'] == time_start_perc) & (am_neq_full_results_pooled_df['time_end_perc'] == 100)]
    apo_closed_slice_results_pooled_df = apo_neq_full_results_pooled_df[(apo_neq_full_results_pooled_df['time_start_perc'] == time_start_perc) & (apo_neq_full_results_pooled_df['time_end_perc'] == 100)]
    nutlin_closed_slice_results_pooled_df = nutlin_neq_full_results_pooled_df[(nutlin_neq_full_results_pooled_df['time_start_perc'] == time_start_perc) & (nutlin_neq_full_results_pooled_df['time_end_perc'] == 100)]

    
    am_pred_ddg = am_open_slice_results_pooled_df["dG pooled (kcal/mol)"] - apo_closed_slice_results_pooled_df["dG pooled (kcal/mol)"]
    nutlin_pred_ddg = nutlin_closed_slice_results_pooled_df["dG pooled (kcal/mol)"] - apo_closed_slice_results_pooled_df["dG pooled (kcal/mol)"]

    am_pred_ddg_std = np.sqrt(am_open_slice_results_pooled_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_pooled_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std = np.sqrt(nutlin_closed_slice_results_pooled_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_pooled_df["dg std. (kcal/mol)"]**2)

    # am_pred_ddg_std, nutlin_pred_ddg_std

    # rename columns
    am_pred_ddg = am_pred_ddg.rename("ddG avg. (kcal/mol)")
    am_pred_ddg_std = am_pred_ddg_std.rename("ddG std. (kcal/mol)")
    nutlin_pred_ddg = nutlin_pred_ddg.rename("ddG avg. (kcal/mol)")
    nutlin_pred_ddg_std = nutlin_pred_ddg_std.rename("ddG std. (kcal/mol)")

    am_pred_ddg_df = pd.DataFrame(am_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    am_pred_ddg_std = pd.DataFrame(am_pred_ddg_std, columns=["ddG std. (kcal/mol)"])
    nutlin_pred_ddg_df = pd.DataFrame(nutlin_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    nutlin_pred_ddg_std = pd.DataFrame(nutlin_pred_ddg_std, columns=["ddG std. (kcal/mol)"])

    # reindex the dataframes to match the experimental data
    am_pred_ddg_df.reindex(am_exp_ddg_kcal_df.index)
    am_pred_ddg_std.reindex(am_exp_ddg_kcal_df.index)

    nutlin_pred_ddg_df.reindex(nutlin_exp_ddg_kcal_df.index)
    nutlin_pred_ddg_std.reindex(nutlin_exp_ddg_kcal_df.index)


    # am_pred_ddg_df, nutlin_pred_ddg_df

    predicted_ddg_df = pd.concat([am_pred_ddg_df, nutlin_pred_ddg_df])
    predicted_ddg_std_df = pd.concat([am_pred_ddg_std, nutlin_pred_ddg_std])

    experimental_ddg_df = pd.concat([am_exp_ddg_kcal_df, nutlin_exp_ddg_kcal_df])

    # predicted_ddg_df, experimental_ddg_df

    # to numpy array
    predicted_ddg = predicted_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    predicted_ddg_std = predicted_ddg_std_df["ddG std. (kcal/mol)"].to_numpy()
    experimental_ddg = experimental_ddg_df["ddG (kcal/mol)"].to_numpy()


    # predicted_ddg, experimental_ddg

    rmse_partial = calculate_rmse(predicted_ddg, experimental_ddg)
    mae_partial = calculate_mae(predicted_ddg, experimental_ddg)

    # rmse_partial, mae_partial

    # convert to numpy arrays for plotting
    am_exp_ddg_kcal_np = am_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    am_pred_ddg_np = am_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    am_pred_ddg_std_np = am_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    nutlin_exp_ddg_kcal_np = nutlin_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_np = nutlin_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_std_np = nutlin_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    temporal_rmse[f"{time_start_perc}-{time_end_perc}"] = rmse_partial
    temporal_mae[f"{time_start_perc}-{time_end_perc}"] = mae_partial
    # temporal_std[f"{time_start_perc}-{time_end_perc}"] = predicted_ddg_std.mean()


    ### PLOT OF Dominant Lid States



    fig, axs = plt.subplots(layout='tight')

    x = np.arange(-5, 10, 1)
    y = np.arange(-5, 10, 1)
    plt.rcParams.update({'font.size': 14})
    ax_x_lim, ax_y_lim = -2, 6
    padding_x = 0.15



    axs.fill_between(x, [i - 1 for i in y], [i + 1 for i in y], alpha=0.2, color='grey')
    axs.fill_between(x, [i - 0.5 for i in y], [i + 0.5 for i in y], alpha=0.2, color='grey')
    axs.errorbar(x=nutlin_exp_ddg_kcal_np, y=nutlin_pred_ddg_np, yerr=nutlin_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#6fc1dcff', ecolor='black', label='Nutlin-3a')
    axs.errorbar(x=am_exp_ddg_kcal_np , y=am_pred_ddg_np, yerr=am_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#F07167', ecolor='black', label='AM-7209')
    axs.legend(loc='upper left')
    axs.set_ylim(ax_x_lim, ax_y_lim)
    axs.set_xlim(ax_x_lim, ax_y_lim)
    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    axs.set_ylabel("Predicted $\Delta \Delta G$ (kcal/mol)")
    axs.set_xlabel("Experimental $\Delta \Delta G$ (kcal/mol)")
    axs.set_box_aspect(1)

    # texts_nutlin = [axs.text(nutlin_exp_ddg_kcal[i]-padding_x, nutlin_pred_ddg_df[i], f"{mutations[i]}", ha='right', va='center') for i in range(len(nutlin_exp_ddg_kcal))]
    # texts_am = [axs.text(am_exp_ddg_kcal[i]-padding_x, am_pred_ddg_df[i], f"{mutations[i]}", ha='right', va='center') for i in range(len(am_exp_ddg_kcal))]



    axs.annotate('V14G', (nutlin_exp_ddg_kcal_np[0], nutlin_pred_ddg_np[0]), size=18, xytext=(-55, -5), textcoords="offset pixels")
    axs.annotate('T16G', (nutlin_exp_ddg_kcal_np[1], nutlin_pred_ddg_np[1]), size=18, xytext=(7, -5), textcoords="offset pixels")
    axs.annotate('Q18G', (nutlin_exp_ddg_kcal_np[2], nutlin_pred_ddg_np[2]), size=18, xytext=(-55, -7), textcoords="offset pixels")
    axs.annotate('I19G', (nutlin_exp_ddg_kcal_np[3], nutlin_pred_ddg_np[3]), size=18, xytext=(7, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (nutlin_exp_ddg_kcal_np[4], nutlin_pred_ddg_np[4]), size=18, xytext=(-100, -5), textcoords="offset pixels")

    axs.annotate('V14G', (am_exp_ddg_kcal_np [0], am_pred_ddg_np[0]), size=18, xytext=(7, -5), textcoords="offset pixels")
    axs.annotate('T16G', (am_exp_ddg_kcal_np [1], am_pred_ddg_np[1]), size=18, xytext=(7, -5), textcoords="offset pixels")
    axs.annotate('Q18G', (am_exp_ddg_kcal_np [2], am_pred_ddg_np[2]), size=18, xytext=(7, -5), textcoords="offset pixels")
    axs.annotate('I19G', (am_exp_ddg_kcal_np [3], am_pred_ddg_np[3]), size=18, xytext=(7, -5), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (am_exp_ddg_kcal_np [4], am_pred_ddg_np[4]), size=18, xytext=(15, -5), textcoords="offset pixels")


    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    # from matplotlib.ticker import FormatStrFormatter

    # axs.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

    # bootstrap statistics
    res_rmse = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='RMSE')
    res_mae = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='MUE')
    res_rho = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='rho')
    res_r2 = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='R2')
    res_ktau = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='KTAU')
    print(res_rmse)

    # append lower and upper bounds to the dictionary
    temporal_rmse_lower[f"{time_start_ns}-{time_end_ns}"] = res_rmse['low']
    temporal_rmse_upper[f"{time_start_ns}-{time_end_ns}"] = res_rmse['high']

    textstr = f"RMSE:        {res_rmse['mle']:.2f} [95%: {res_rmse['low']:.2f}, {res_rmse['high']:.2f}]\nMAE:        {res_mae['mle']:.2f} [95%: {res_mae['low']:.2f}, {res_mae['high']:.2f}]\n rho:        {res_rho['mle']:.2f} [95%: {res_rho['low']:.2f}, {res_rho['high']:.2f}]\n R2:        {res_r2['mle']:.2f} [95%: {res_r2['low']:.2f}, {res_r2['high']:.2f}]\nKTAU:        {res_ktau['mle']:.2f} [95%: {res_ktau['low']:.2f}, {res_ktau['high']:.2f}]"
    # full_title = f"{textstr} {title}"
    full_title = f"{textstr}"
    axs.set_title(full_title, family="monospace", loc='right')
    fig.set_size_inches(10,10)
    # plt.savefig(f'paper_neq_fep_plots/{time_start_perc}-{time_end_perc}_perc.pdf', dpi=600, bbox_inches='tight')

## FF99SB-ILDN + OPC Analysis

In [ ]:
am_df_full = pd.read_csv('eq_fep_analysed_results_10_to_100_ff99sb/am_df_full.csv')
apo_df_full = pd.read_csv('eq_fep_analysed_results_10_to_100_ff99sb/apo_df_full.csv')
nutlin_df_full = pd.read_csv('eq_fep_analysed_results_10_to_100_ff99sb/nutlin_df_full.csv')

In [ ]:
am_df_full

### Plotting

In [ ]:
sns.set_theme(style="ticks")
sns.set_context("paper", font_scale=2)

temporal_rmse = {}
temporal_rmse_lower = {}
temporal_rmse_upper = {}
temporal_mae = {}
temporal_std = {}

time_start_ns = 10 # ns

for time_end_ns in [100]:
    time_end = time_end_ns * 1000 # ns to ps

    am_open_slice_results_df = am_df_full[(am_df_full['time_start'] == 10000) & (am_df_full['time_end'] == time_end)]
    apo_closed_slice_results_df = apo_df_full[(apo_df_full['time_start'] == 10000) & (apo_df_full['time_end'] == time_end)]
    nutlin_closed_slice_results_df = nutlin_df_full[(nutlin_df_full['time_start'] == 10000) & (nutlin_df_full['time_end'] == time_end)]



    time_end = time_end_ns * 1000 # ns to ps
    am_pred_ddg = am_open_slice_results_df["dG avg. (kcal/mol)"] - apo_closed_slice_results_df["dG avg. (kcal/mol)"]
    nutlin_pred_ddg = nutlin_closed_slice_results_df["dG avg. (kcal/mol)"] - apo_closed_slice_results_df["dG avg. (kcal/mol)"]

    am_pred_ddg_std = np.sqrt(am_open_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)
    nutlin_pred_ddg_std = np.sqrt(nutlin_closed_slice_results_df["dg std. (kcal/mol)"]**2 + apo_closed_slice_results_df["dg std. (kcal/mol)"]**2)

    # am_pred_ddg_std, nutlin_pred_ddg_std

    # rename columns
    am_pred_ddg = am_pred_ddg.rename("ddG avg. (kcal/mol)")
    am_pred_ddg_std = am_pred_ddg_std.rename("ddG std. (kcal/mol)")
    nutlin_pred_ddg = nutlin_pred_ddg.rename("ddG avg. (kcal/mol)")
    nutlin_pred_ddg_std = nutlin_pred_ddg_std.rename("ddG std. (kcal/mol)")

    print(am_pred_ddg, am_pred_ddg_std)

    am_pred_ddg_df = pd.DataFrame(am_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    am_pred_ddg_std = pd.DataFrame(am_pred_ddg_std, columns=["ddG std. (kcal/mol)"])
    nutlin_pred_ddg_df = pd.DataFrame(nutlin_pred_ddg, columns=["ddG avg. (kcal/mol)"])
    nutlin_pred_ddg_std = pd.DataFrame(nutlin_pred_ddg_std, columns=["ddG std. (kcal/mol)"])

    # reindex the dataframes to match the experimental data
    am_pred_ddg_df.reindex(am_exp_ddg_kcal_df.index)
    am_pred_ddg_std.reindex(am_exp_ddg_kcal_df.index)

    nutlin_pred_ddg_df.reindex(nutlin_exp_ddg_kcal_df.index)
    nutlin_pred_ddg_std.reindex(nutlin_exp_ddg_kcal_df.index)


    # am_pred_ddg_df, nutlin_pred_ddg_df

    predicted_ddg_df = pd.concat([am_pred_ddg_df, nutlin_pred_ddg_df])
    predicted_ddg_std_df = pd.concat([am_pred_ddg_std, nutlin_pred_ddg_std])

    experimental_ddg_df = pd.concat([am_exp_ddg_kcal_df, nutlin_exp_ddg_kcal_df])

    # predicted_ddg_df, experimental_ddg_df

    # to numpy array
    predicted_ddg = predicted_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    predicted_ddg_std = predicted_ddg_std_df["ddG std. (kcal/mol)"].to_numpy()
    experimental_ddg = experimental_ddg_df["ddG (kcal/mol)"].to_numpy()


    # predicted_ddg, experimental_ddg

    rmse_partial = calculate_rmse(predicted_ddg, experimental_ddg)
    mae_partial = calculate_mae(predicted_ddg, experimental_ddg)

    # rmse_partial, mae_partial

    # convert to numpy arrays for plotting
    am_exp_ddg_kcal_np = am_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    am_pred_ddg_np = am_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    am_pred_ddg_std_np = am_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    nutlin_exp_ddg_kcal_np = nutlin_exp_ddg_kcal_df["ddG (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_np = nutlin_pred_ddg_df["ddG avg. (kcal/mol)"].to_numpy()
    nutlin_pred_ddg_std_np = nutlin_pred_ddg_std["ddG std. (kcal/mol)"].to_numpy()

    temporal_rmse[f"{time_start_ns}-{time_end_ns}"] = rmse_partial
    temporal_mae[f"{time_start_ns}-{time_end_ns}"] = mae_partial
    temporal_std[f"{time_start_ns}-{time_end_ns}"] = predicted_ddg_std.mean()


    ### PLOT OF Dominant Lid States



    fig, axs = plt.subplots(layout='tight')

    x = np.arange(-5, 10, 1)
    y = np.arange(-5, 10, 1)
    plt.rcParams.update({'font.size': 14})
    ax_x_lim, ax_y_lim = -2, 6
    padding_x = 0.15



    axs.fill_between(x, [i - 1 for i in y], [i + 1 for i in y], alpha=0.2, color='grey')
    axs.fill_between(x, [i - 0.5 for i in y], [i + 0.5 for i in y], alpha=0.2, color='grey')
    axs.errorbar(x=nutlin_exp_ddg_kcal_np, y=nutlin_pred_ddg_np, yerr=nutlin_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#6fc1dcff', ecolor='black', label='Nutlin-3a')
    axs.errorbar(x=am_exp_ddg_kcal_np , y=am_pred_ddg_np, yerr=am_pred_ddg_std_np, fmt='o', mec='black', capsize=6, ms=9, color='#F07167', ecolor='black', label='AM-7209')
    axs.legend(loc='upper left')
    axs.set_ylim(ax_x_lim, ax_y_lim)
    axs.set_xlim(ax_x_lim, ax_y_lim)
    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))
    axs.set_ylabel("Predicted $\Delta \Delta G$ (kcal/mol)")
    axs.set_xlabel("Experimental $\Delta \Delta G$ (kcal/mol)")
    axs.set_box_aspect(1)




    axs.annotate('V14G', (nutlin_exp_ddg_kcal_np[0], nutlin_pred_ddg_np[0]), size=18, xytext=(-50, -15), textcoords="offset pixels")
    axs.annotate('T16G', (nutlin_exp_ddg_kcal_np[1], nutlin_pred_ddg_np[1]), size=18, xytext=(10, -5), textcoords="offset pixels")
    axs.annotate('Q18G', (nutlin_exp_ddg_kcal_np[2], nutlin_pred_ddg_np[2]), size=18, xytext=(-55, -2), textcoords="offset pixels")
    axs.annotate('I19G', (nutlin_exp_ddg_kcal_np[3], nutlin_pred_ddg_np[3]), size=18, xytext=(7, -8), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (nutlin_exp_ddg_kcal_np[4], nutlin_pred_ddg_np[4]), size=18, xytext=(-100, -8), textcoords="offset pixels")

    axs.annotate('V14G', (am_exp_ddg_kcal_np [0], am_pred_ddg_np[0]), size=18, xytext=(7, -3), textcoords="offset pixels")
    axs.annotate('T16G', (am_exp_ddg_kcal_np [1], am_pred_ddg_np[1]), size=18, xytext=(7, -5), textcoords="offset pixels")
    axs.annotate('Q18G', (am_exp_ddg_kcal_np [2], am_pred_ddg_np[2]), size=18, xytext=(10, -3), textcoords="offset pixels")
    axs.annotate('I19G', (am_exp_ddg_kcal_np [3], am_pred_ddg_np[3]), size=18, xytext=(10, -6), textcoords="offset pixels")
    axs.annotate('T16G-I19G', (am_exp_ddg_kcal_np [4], am_pred_ddg_np[4]), size=18, xytext=(10, 2), textcoords="offset pixels")

    axs.xaxis.set_minor_locator(MultipleLocator(0.1))
    axs.yaxis.set_minor_locator(MultipleLocator(0.1))

    # bootstrap statistics
    res_rmse = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='RMSE')
    res_mae = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='MUE')
    res_rho = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='rho')
    res_r2 = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='R2')
    res_ktau = stats.bootstrap_statistic(y_true=experimental_ddg, y_pred=predicted_ddg, dy_pred=predicted_ddg_std, statistic='KTAU')
    print(res_rmse)

    # append lower and upper bounds to the dictionary
    temporal_rmse_lower[f"{time_start_ns}-{time_end_ns}"] = res_rmse['low']
    temporal_rmse_upper[f"{time_start_ns}-{time_end_ns}"] = res_rmse['high']

    textstr = f"RMSE:        {res_rmse['mle']:.2f} [95%: {res_rmse['low']:.2f}, {res_rmse['high']:.2f}]\nMAE:        {res_mae['mle']:.2f} [95%: {res_mae['low']:.2f}, {res_mae['high']:.2f}]\n rho:        {res_rho['mle']:.2f} [95%: {res_rho['low']:.2f}, {res_rho['high']:.2f}]\n R2:        {res_r2['mle']:.2f} [95%: {res_r2['low']:.2f}, {res_r2['high']:.2f}]\nKTAU:        {res_ktau['mle']:.2f} [95%: {res_ktau['low']:.2f}, {res_ktau['high']:.2f}]"
    #full_title = f"{textstr} {title}"
    full_title = f"{textstr}"
    axs.set_title(full_title, family="monospace", loc='right')
    fig.set_size_inches(10,10)
    # plt.savefig(f'paper_eq_fep_plots/{time_start_ns}-{time_end_ns}ns_ff99sb.png', dpi=600, bbox_inches='tight')
    # plt.savefig(f'paper_eq_fep_plots/{time_start_ns}-{time_end_ns}ns_ff99sb.pdf', dpi=600, bbox_inches='tight')

## Joint Full Analysis

In [ ]:
# plot with lower and upper bounds
from matplotlib.colors import rgb2hex

sns.set_style("ticks")
sns.set_context("paper", font_scale=1.5)


eq_temporal_rmse_forward_df = pd.read_csv('eq_fep_analysed_results_0_to_100_statistics/temporal_rmse_df.csv')
eq_temporal_rmse_forward_lower_df = pd.read_csv('eq_fep_analysed_results_0_to_100_statistics/temporal_rmse_lower_df.csv')
eq_temporal_rmse_forward_upper_df = pd.read_csv('eq_fep_analysed_results_0_to_100_statistics/temporal_rmse_upper_df.csv')

neq_temporal_rmse_forward_df = pd.read_csv('neq_fep_analysed_results_0_to_100_statistics/temporal_rmse_df.csv')
neq_temporal_rmse_forward_lower_df = pd.read_csv('neq_fep_analysed_results_0_to_100_statistics/temporal_rmse_lower_df.csv')
neq_temporal_rmse_forward_upper_df = pd.read_csv('neq_fep_analysed_results_0_to_100_statistics/temporal_rmse_upper_df.csv')

neq_temporal_pooled_rmse_forward_df = pd.read_csv('neq_fep_analysed_results_0_to_100_statistics/temporal_rmse_pooled_df.csv')
neq_temporal_pooled_rmse_forward_lower_df = pd.read_csv('neq_fep_analysed_results_0_to_100_statistics/temporal_rmse_lower_pooled_df.csv')
neq_temporal_pooled_rmse_forward_upper_df = pd.read_csv('neq_fep_analysed_results_0_to_100_statistics/temporal_rmse_upper_pooled_df.csv')

eq_temporal_rmse_reverse_df = pd.read_csv('eq_fep_analysed_results_100_to_10_statistics/temporal_rmse_df.csv')
eq_temporal_rmse_reverse_lower_df = pd.read_csv('eq_fep_analysed_results_100_to_10_statistics/temporal_rmse_lower_df.csv')
eq_temporal_rmse_reverse_upper_df = pd.read_csv('eq_fep_analysed_results_100_to_10_statistics/temporal_rmse_upper_df.csv')

neq_temporal_rmse_reverse_df = pd.read_csv('neq_fep_analysed_results_100_to_10_statistics/temporal_rmse_df.csv')
neq_temporal_rmse_reverse_lower_df = pd.read_csv('neq_fep_analysed_results_100_to_10_statistics/temporal_rmse_lower_df.csv')
neq_temporal_rmse_reverse_upper_df = pd.read_csv('neq_fep_analysed_results_100_to_10_statistics/temporal_rmse_upper_df.csv')

neq_temporal_pooled_rmse_reverse_df = pd.read_csv('neq_fep_analysed_results_100_to_10_statistics/temporal_rmse_pooled_df.csv')
neq_temporal_pooled_rmse_reverse_lower_df = pd.read_csv('neq_fep_analysed_results_100_to_10_statistics/temporal_rmse_lower_pooled_df.csv')
neq_temporal_pooled_rmse_reverse_upper_df = pd.read_csv('neq_fep_analysed_results_100_to_10_statistics/temporal_rmse_upper_pooled_df.csv')

#temporal_rmse_forward_df, temporal_rmse_forward_lower_df, temporal_rmse_forward_upper_df, temporal_rmse_reverse_df, temporal_rmse_reverse_lower_df, temporal_rmse_reverse_upper_df

# merge the dataframes
eq_temporal_rmse_df = pd.concat([eq_temporal_rmse_forward_df, eq_temporal_rmse_reverse_df], axis=0)
eq_temporal_rmse_lower_df = pd.concat([eq_temporal_rmse_forward_lower_df, eq_temporal_rmse_reverse_lower_df], axis=0)
eq_temporal_rmse_upper_df = pd.concat([eq_temporal_rmse_forward_upper_df, eq_temporal_rmse_reverse_upper_df], axis=0)

neq_temporal_rmse_df = pd.concat([neq_temporal_rmse_forward_df, neq_temporal_rmse_reverse_df], axis=0)
neq_temporal_rmse_lower_df = pd.concat([neq_temporal_rmse_forward_lower_df, neq_temporal_rmse_reverse_lower_df], axis=0)
neq_temporal_rmse_upper_df = pd.concat([neq_temporal_rmse_forward_upper_df, neq_temporal_rmse_reverse_upper_df], axis=0)

neq_temporal_pooled_rmse_df = pd.concat([neq_temporal_pooled_rmse_forward_df, neq_temporal_pooled_rmse_reverse_df], axis=0)
neq_temporal_pooled_rmse_lower_df = pd.concat([neq_temporal_pooled_rmse_forward_lower_df, neq_temporal_pooled_rmse_reverse_lower_df], axis=0)
neq_temporal_pooled_rmse_upper_df = pd.concat([neq_temporal_pooled_rmse_forward_upper_df, neq_temporal_pooled_rmse_reverse_upper_df], axis=0)

# rename the columns
eq_temporal_rmse_df.columns = ["Sampling slice (%)", "RMSE (kcal/mol)"]
eq_temporal_rmse_lower_df.columns = ["Sampling slice (%)", "RMSE Lower (kcal/mol)"]
eq_temporal_rmse_upper_df.columns = ["Sampling slice (%)", "RMSE Upper (kcal/mol)"]

neq_temporal_rmse_df.columns = ["Sampling slice (%)", "RMSE (kcal/mol)"]
neq_temporal_rmse_lower_df.columns = ["Sampling slice (%)", "RMSE Lower (kcal/mol)"]
neq_temporal_rmse_upper_df.columns = ["Sampling slice (%)", "RMSE Upper (kcal/mol)"]

neq_temporal_pooled_rmse_df.columns = ["Sampling slice (%)", "RMSE (kcal/mol)"]
neq_temporal_pooled_rmse_lower_df.columns = ["Sampling slice (%)", "RMSE Lower (kcal/mol)"]
neq_temporal_pooled_rmse_upper_df.columns = ["Sampling slice (%)", "RMSE Upper (kcal/mol)"]


eq_temporal_rmse_df = eq_temporal_rmse_df.pivot_table(values='RMSE (kcal/mol)', columns='Sampling slice (%)', sort=False)
eq_temporal_rmse_lower_df = eq_temporal_rmse_lower_df.pivot_table(values='RMSE Lower (kcal/mol)', columns='Sampling slice (%)', sort=False)
eq_temporal_rmse_upper_df = eq_temporal_rmse_upper_df.pivot_table(values='RMSE Upper (kcal/mol)', columns='Sampling slice (%)', sort=False)

neq_temporal_rmse_df = neq_temporal_rmse_df.pivot_table(values='RMSE (kcal/mol)', columns='Sampling slice (%)', sort=False)
neq_temporal_rmse_lower_df = neq_temporal_rmse_lower_df.pivot_table(values='RMSE Lower (kcal/mol)', columns='Sampling slice (%)', sort=False)
neq_temporal_rmse_upper_df = neq_temporal_rmse_upper_df.pivot_table(values='RMSE Upper (kcal/mol)', columns='Sampling slice (%)', sort=False)

neq_temporal_pooled_rmse_df = neq_temporal_pooled_rmse_df.pivot_table(values='RMSE (kcal/mol)', columns='Sampling slice (%)', sort=False)
neq_temporal_pooled_rmse_lower_df = neq_temporal_pooled_rmse_lower_df.pivot_table(values='RMSE Lower (kcal/mol)', columns='Sampling slice (%)', sort=False)
neq_temporal_pooled_rmse_upper_df = neq_temporal_pooled_rmse_upper_df.pivot_table(values='RMSE Upper (kcal/mol)', columns='Sampling slice (%)', sort=False)


# reverse the palette
aerospace_orange = "#FF4F00"

# plot a line plot with the lower and upper bounds
fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(20, 7), sharey=True,)

axs[0].plot(eq_temporal_rmse_df.T, alpha=1, color=aerospace_orange, linewidth=2)
axs[0].plot(eq_temporal_rmse_upper_df.T, alpha=0.0,)
axs[0].plot(eq_temporal_rmse_lower_df.T, alpha=0.0)

line = axs[0].get_lines()
axs[0].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), alpha=.4, color=aerospace_orange)

# add a grey horizontal line at 1
axs[0].axhline(y=1, color='grey', linestyle='--')
axs[0].margins(x=0.0)
axs[0].tick_params(axis='x', labelrotation=90)


axs[1].plot(neq_temporal_rmse_df.T, alpha=1, color=aerospace_orange, linewidth=2)
axs[1].plot(neq_temporal_rmse_upper_df.T, alpha=0.0,)
axs[1].plot(neq_temporal_rmse_lower_df.T, alpha=0.0)

line = axs[1].get_lines()
axs[1].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), alpha=.4, color=aerospace_orange)
axs[1].axhline(y=1, color='grey', linestyle='--')
axs[1].margins(x=0.0)
axs[1].tick_params(axis='x', labelrotation=90)


axs[2].plot(neq_temporal_pooled_rmse_df.T, alpha=1, color=aerospace_orange, linewidth=2)
axs[2].plot(neq_temporal_pooled_rmse_upper_df.T, alpha=0.0,)
axs[2].plot(neq_temporal_pooled_rmse_lower_df.T, alpha=0.0)

line = axs[2].get_lines()
axs[2].fill_between(line[1].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), alpha=.4, color=aerospace_orange)
axs[2].axhline(y=1, color='grey', linestyle='--')
axs[2].margins(x=0.0)
axs[2].tick_params(axis='x', labelrotation=90)


# add axis labels
axs[0].set_ylabel("ΔΔG RMSE (kcal/mol)")
axs[0].set_xlabel("Portion of Total Sampling Time (%)")
axs[1].set_xlabel("Portion of Total Sampling Time (%)")
axs[2].set_xlabel("Portion of Total Sampling Time (%)")


axs[0].set_title("EQ", fontsize=25, font='monospace')
axs[1].set_title("NEQ", fontsize=25, font='monospace')
axs[2].set_title("NEQ Pooled", fontsize=16, font='monospace')

# fix y-limit to 0
axs[0].set_ylim(0, axs[0].get_ylim()[1])

# plt.tight_layout()

# plt.savefig('paper_combined_fep_plots/combined_temporal_rmse_lineplot.png', dpi=300, bbox_inches='tight')


In [ ]:
eq_am_std_forward_df = pd.read_csv('eq_fep_analysed_results_0_to_100_statistics/temporal_am_pred_ddg_std_df.csv')
eq_am_std_reverse_df = pd.read_csv('eq_fep_analysed_results_100_to_10_statistics/temporal_am_pred_ddg_std_df.csv')
eq_nutlin_std_forward_df = pd.read_csv('eq_fep_analysed_results_0_to_100_statistics/temporal_nutlin_pred_ddg_std_df.csv')
eq_nutlin_std_reverse_df = pd.read_csv('eq_fep_analysed_results_100_to_10_statistics/temporal_nutlin_pred_ddg_std_df.csv')

neq_am_std_forward_df = pd.read_csv('neq_fep_analysed_results_0_to_100_statistics/temporal_am_pred_ddg_std_df.csv')
neq_am_std_reverse_df = pd.read_csv('neq_fep_analysed_results_100_to_10_statistics/temporal_am_pred_ddg_std_df.csv')
neq_nutlin_std_forward_df = pd.read_csv('neq_fep_analysed_results_0_to_100_statistics/temporal_nutlin_pred_ddg_std_df.csv')
neq_nutlin_std_reverse_df = pd.read_csv('neq_fep_analysed_results_100_to_10_statistics/temporal_nutlin_pred_ddg_std_df.csv')

# merge the dataframes
eq_am_std_df = pd.concat([eq_am_std_forward_df, eq_am_std_reverse_df], axis=0)
eq_nutlin_std_df = pd.concat([eq_nutlin_std_forward_df, eq_nutlin_std_reverse_df], axis=0)
neq_am_std_df = pd.concat([neq_am_std_forward_df, neq_am_std_reverse_df], axis=0)
neq_nutlin_std_df = pd.concat([neq_nutlin_std_forward_df, neq_nutlin_std_reverse_df], axis=0)

# convert values to percentages for eq only, since the neq values are already in percentages
eq_am_std_df['time_start_perc'] = eq_am_std_df['time_start'] / 1000
eq_am_std_df['time_end_perc'] = eq_am_std_df['time_end'] / 1000

eq_nutlin_std_df['time_start_perc'] = eq_nutlin_std_df['time_start'] / 1000
eq_nutlin_std_df['time_end_perc'] = eq_nutlin_std_df['time_end'] / 1000

In [ ]:
# combine the dataframes
combined_eq_std_df = pd.concat([eq_am_std_df, eq_nutlin_std_df], axis=0)
combined_neq_std_df = pd.concat([neq_am_std_df, neq_nutlin_std_df], axis=0)

# convert column values to int, then string
combined_eq_std_df['time_start_perc'] = combined_eq_std_df['time_start_perc'].astype(int)
combined_eq_std_df['time_end_perc'] = combined_eq_std_df['time_end_perc'].astype(int)
combined_eq_std_df['time_start_perc'] = combined_eq_std_df['time_start_perc'].astype(str)
combined_eq_std_df['time_end_perc'] = combined_eq_std_df['time_end_perc'].astype(str)

combined_neq_std_df['time_start_perc'] = combined_neq_std_df['time_start_perc'].astype(int)
combined_neq_std_df['time_end_perc'] = combined_neq_std_df['time_end_perc'].astype(int)
combined_neq_std_df['time_start_perc'] = combined_neq_std_df['time_start_perc'].astype(str)
combined_neq_std_df['time_end_perc'] = combined_neq_std_df['time_end_perc'].astype(str)

# combine values from two columns
combined_eq_std_df['Sampling slice (%)'] = combined_eq_std_df['time_start_perc'] + "-" + combined_eq_std_df['time_end_perc']
combined_neq_std_df['Sampling slice (%)'] = combined_neq_std_df['time_start_perc'] + "-" + combined_neq_std_df['time_end_perc']


In [ ]:
# change mutant names to uppercase
combined_eq_std_df['mutant'] = combined_eq_std_df['mutant'].str.upper()
combined_neq_std_df['mutant'] = combined_neq_std_df['mutant'].str.upper()

# for T16G_I19G, change to T16G-I19G
combined_eq_std_df.loc[combined_eq_std_df["mutant"] == "T16G_I19G", "mutant"] = "T16G-I19G"
combined_neq_std_df.loc[combined_neq_std_df["mutant"] == "T16G_I19G", "mutant"] = "T16G-I19G"


In [ ]:
grape = "#6F2DA8"

sns.set_style("ticks")
sns.set_context("paper", font_scale=1.5)


palette = sns.color_palette("Dark2", 5)

fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(20, 7), sharey=True)


sns.lineplot(data=combined_eq_std_df, x=combined_eq_std_df['Sampling slice (%)'], y=combined_eq_std_df['ddg std. (kcal/mol)'], hue=combined_eq_std_df['mutant'], ax=axs[0], palette=palette, marker='o', linestyle='--', errorbar=None)
sns.lineplot(data=combined_eq_std_df, x=combined_eq_std_df['Sampling slice (%)'], y=combined_eq_std_df['ddg std. (kcal/mol)'], hue=None, ax=axs[0], color=grape, linewidth=2, errorbar=None, label="Combined")
sns.lineplot(data=combined_neq_std_df, x=combined_neq_std_df['Sampling slice (%)'], y=combined_neq_std_df['ddg std. (kcal/mol)'], hue=combined_neq_std_df['mutant'], ax=axs[1], palette=palette, marker='o', linestyle='--', errorbar=None)
sns.lineplot(data=combined_neq_std_df, x=combined_neq_std_df['Sampling slice (%)'], y=combined_neq_std_df['ddg std. (kcal/mol)'], hue=None, ax=axs[1], color=grape, linewidth=2, errorbar=None, label="Combined")


# add axis labels
axs[0].set_ylabel("ΔΔG Mean SD (kcal/mol)")
axs[0].set_xlabel("Portion of Total Sampling Time (%)")
axs[1].set_xlabel("Portion of Total Sampling Time (%)")


axs[0].legend(loc='upper left', frameon=False)
axs[1].legend(loc='upper left', frameon=False)

axs[0].tick_params(axis='x', labelrotation=90)
axs[1].tick_params(axis='x', labelrotation=90)

axs[0].set_title("EQ", fontsize=16, font='monospace')
axs[1].set_title("NEQ", fontsize=16,font='monospace')

# fix y-limit to 0
axs[0].set_ylim(0, axs[0].get_ylim()[1])

# plt.tight_layout()
# plt.savefig('paper_combined_fep_plots/combined_temporal_std_lineplot.png', dpi=300, bbox_inches='tight')

## NEQ Overlap Distribution Analysis

In [ ]:
apo_neq_full_results_df = pd.DataFrame()

for mutant in ["v14g", "t16g", "q18g", "i19g", "i19g_1ns", "i61g", "i99g", "t16g_i19g"]:
    print(f"Processing mutant: {mutant}")

    time_start = 0
    time_end_slice = [2750]


    for time_end in time_end_slice:

        apo_closed_df = pd.read_csv(f"neq_xvg_analysis/apo_closed/system_results/results_0_{time_end}_mod.dat",
                        names=["mutant", "dG (kcal/mol)"])

        # Define the string to search for
        search_string = rf'\b{mutant}_repl_\d+\b'

        # Apply the string containment check across all elements
        contains_string = apo_closed_df.applymap(lambda x: specific_pattern_match(x, search_string))

        # Filter the DataFrame to get only the rows where any element contains the string
        filtered_df = apo_closed_df[contains_string.any(axis=1)]

        # Use a regular expression to extract floating point values
        # The regex pattern r'([-+]?\d*\.\d+|\d+)' matches both integers and floating point numbers
        filtered_df['extracted dG (kcal/mol)'] = filtered_df["dG (kcal/mol)"].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)
        print(filtered_df)


        if len(filtered_df['extracted dG (kcal/mol)']) != 5 or filtered_df['extracted dG (kcal/mol)'].isnull().sum() > 0:
            raise ValueError("The number of extracted values is not equal to 5")

        # Display the DataFrame with the extracted values
        mean_dg = filtered_df['extracted dG (kcal/mol)'].mean()
        std_dg = filtered_df['extracted dG (kcal/mol)'].std()
        time_end_perc = time_end / time_end_slice[-1] * 100
        processed_df = pd.DataFrame.from_dict({"mutant": mutant, "dG avg. (kcal/mol)": mean_dg, "dg std. (kcal/mol)": std_dg, "time_start_perc": time_start, "time_end_perc": time_end_perc}, orient="index").T
        apo_neq_full_results_df = pd.concat([apo_neq_full_results_df, processed_df])
        apo_neq_full_results_df.reset_index(drop=True, inplace=True)

In [ ]:
sns.set_style("ticks")
sns.set_context("paper", font_scale=1.5)

for mutant in ["v14g", "t16g", "q18g", "i19g", "i19g_1ns", "i61g", "i99g",  "t16g_i19g"]:

    std_df = apo_neq_full_results_df[(apo_neq_full_results_df['time_start_perc'] == 0) & (apo_neq_full_results_df['time_end_perc'] == 100)]
    std_df = std_df[std_df['mutant'] == mutant]
    std_mut = std_df['dg std. (kcal/mol)'].values[0]
    std_mut = std_mut.round(2)


    work_dir = f"neq_xvg_analysis/apo_closed/system_pooled_results/{mutant}/results_0_to_2750_pooled"

    forward_work_df = pd.read_csv(f"{work_dir}/integA.dat", sep="\s+", names=["Snapshot", "Work (kJ/mol)"])
    forward_work_df["Direction"] = "forward"

    # convert the work to kcal/mol
    forward_work_df["Work (kcal/mol)"] = arr_to_kJ2kcal(forward_work_df["Work (kJ/mol)"]) 

    reverse_work_df = pd.read_csv(f"{work_dir}/integB.dat", sep="\s+", names=["Snapshot", "Work (kJ/mol)"])
    reverse_work_df["Direction"] = "reverse"


    reverse_work_df["Work (kcal/mol)"] = arr_to_kJ2kcal(reverse_work_df["Work (kJ/mol)"]) 

    # accesss each frame and split the frame number the full path to the frame
    forward_work_df["Snapshot"] = forward_work_df["Snapshot"].apply(lambda x: x.split("/")[-1].split(".")[0])
    reverse_work_df["Snapshot"] = reverse_work_df["Snapshot"].apply(lambda x: x.split("/")[-1].split(".")[0])

    forward_work_df

    fig, ax1 = plt.subplots(figsize=(10, 5))

    ax1 = sns.histplot(data=forward_work_df, x='Work (kcal/mol)', stat="density", bins=40, kde=True, ax=ax1, label="Forward", color="#6F2DA8", alpha=0.7, edgecolor='black')
    ax1 = sns.histplot(data=reverse_work_df, x='Work (kcal/mol)', stat="density", bins=40, kde=True, ax=ax1, label="Reverse", color="#FF4F00", alpha=0.7, edgecolor='black')
    ax1.set_xlabel("Work (kcal/mol)")
    ax1.text(0.5, 0.9, f'SD:{std_mut} kcal/mol', horizontalalignment='center', verticalalignment='center', transform=ax1.transAxes, font="monospace", bbox=dict(edgecolor="black", fill=False, alpha=1))

    # set title
    if mutant == "i19g_1ns":
        mutant_title = "i19g (1ns)"
    elif mutant == "t16g_i19g":
        mutant_title = "t16g-i19g"
    else:
        mutant_title = mutant
    ax1.set_title(f"{mutant_title.upper()}", font="monospace")
    ax1.legend(loc='upper left', edgecolor='black')